<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

This notebook should be run using Default Spark 3.3.x & Python 3.10 runtime environment. If you are viewing this in Watson Studio and do not see Python 3.10.x with Spark in the upper right corner of your screen, please update the runtime now. It requires service credentials for the following services:

<li>Watson OpenScale
<li>Watson Machine Learning
<li>Db2

## Credentials for IBM Cloud Pak for Data
In the following code boxes, you need to replace the sample data with your own credentials. You can acquire the information from your system administrator or through the Cloud Pak for Data dashboard.

### Obtaining your Watson OpenScale credentials
You can retrieve the URL by running the following command: `oc get route -n namespace1 --no-headers | awk '{print $2}'`
Replace the `namespace1` variable with your namespace.

You should have been assigned a username and password when you were added to the Cloud Pak for Data system. You might need to ask either your database administrator or your system administrator for some of the information.

In [1]:
############################################################################################
# Paste your Watson OpenScale credentials into the following section and then run this cell.
############################################################################################
WOS_CREDENTIALS = {
      "url": "<CP4D cluster URL>",
      "username": "*****",
      "password": "*****"
  }

### Your Watson OpenScale GUID
For most systems, the default GUID is already entered for you. You would only need to update this particular entry if the GUID was changed from the default.

In [2]:
WOS_GUID="00000000-0000-0000-0000-000000000000"

### Your Watson OpenScale GUID
For most systems, the default GUID is already entered for you. You would only need to update this particular entry if the GUID was changed from the default.

In [3]:
WML_CREDENTIALS = WOS_CREDENTIALS.copy()
WML_CREDENTIALS['instance_id']='openshift'
WML_CREDENTIALS['version']='4.7' #If your env is CP4D 4.x.x then specify "4.x.x" instead of "4.7"
WML_CREDENTIALS

### Your database credentials and schema
Normally, you must obtain the database credentials and schema name from your database administrator, however, if you deployed one of the database options from the Cloud Pak for Data UI, you should be able to retrieve credentials yourself by going to My data and right-clicking on the database tile.

In [4]:
####################################################################################
# Paste your database credentials into the following section and then run this cell.
####################################################################################
DATABASE_CREDENTIALS = {
      "db": "SAMPLE",
      "db_type": "db2",
      "hostname": "<host>",
      "password": "******",
      "port": "<port>",
      "username": "*****"
  }

In [5]:
SCHEMA_NAME = "<schema_name>"

## Integration system credentials (IBM OpenPages MRG) 
As part of the closed beta, you were sent an email message from IBM that provides the URL, username, and password of an IBM OpenPages system that you can use. Replace the following credentials with the ones that you received in that email message. Please make sure to provide the location of Openpages instance as "location" and fill it with "ibmcloud" or "cp4d". By default the value for location will be "cp4d".

In [6]:
#########################################################################################
# Paste your IBM OpenPages credentials into the following section and then run this cell.
#########################################################################################
OPENPAGES_CREDENTIALS = {
     "url": "<Openpages URL>",
     "username": "*****",
     "password": "*****"
     # "location": "*****"
  }

## Model ID
For the model that you create, copy the `model_id` from the OpenPages Model URL from the browser address bar. For example, if you create a model and view it in the OpenPages UI, you might see a URL such as "https://mrgbeta.op-ibm.com/app/jspview/react/grc/task-view/7916?". In this example, the `model_id` is **7916**. Paste that value into the following code cell.

In [7]:
####################################################################
# Paste your model ID in the following field and then run this cell.
####################################################################
openpages_model_id="<model_id>"

## Package installation
The following opensource packages must be installed into this notebook instance so that they are available to use during processing.

In [ ]:
!rm -rf /home/spark/shared/user-libs/python3.6*

#If you are running this notebook in non IBM Watson Studio env then uncomment the below pip statements and run it
#!pip install --upgrade pyspark==3.3.0  --no-cache | tail -n 1
#!pip install numpy==1.20.1 --no-cache | tail -n 1
#!pip install scikit-learn==1.1 --no-cache | tail -n 1
#!pip install --upgrade pandas==1.2.3 --no-cache | tail -n 1
#!pip install --upgrade SciPy --no-cache | tail -n 1

!pip install --upgrade ibm-watson-machine-learning --user | tail -n 1
!pip install --upgrade ibm_watson_openscale --no-cache | tail -n 1

## Load the training data from Github
So you don't have to manually generate training data, we've provided a sample and placed it in a publicly available Github repo.

In [8]:
!rm german_credit_data_biased_training.csv
!wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/german_credit_data_biased_training.csv

--2024-08-08 05:50:34--  https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/german_credit_data_biased_training.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689622 (673K) [text/plain]
Saving to: ‘german_credit_data_biased_training.csv’

german_credit_data_ 100%[===================>] 673.46K  --.-KB/s    in 0.09s   

2024-08-08 05:50:34 (7.19 MB/s) - ‘german_credit_data_biased_training.csv’ saved [689622/689622]



In [9]:
import json
import requests
import base64
from requests.auth import HTTPBasicAuth
import time

# Deploy the models

The following cells deploy both the pre-prod and challenger models into the Watson Machine Learning instance.

In [10]:
PRE_PROD_MODEL_NAME="German Credit Risk Model - PreProd"
PRE_PROD_DEPLOYMENT_NAME="German Credit Risk Model - PreProd"

PRE_PROD_CHALLENGER_MODEL_NAME="German Credit Risk Model - Challenger"
PRE_PROD_CHALLENGER_DEPLOYMENT_NAME="German Credit Risk Model - Challenger"

PRE_PROD_SPACE_NAME="wml_preprod"

## Initialize WML client and set the space_id parameter
Analytic spaces are used to categorize assets and can be associated with a project inside of Watson Studio.

In [11]:
from ibm_watson_machine_learning import APIClient
wml_client = APIClient(WML_CREDENTIALS)
print(wml_client.service_instance.get_url())

wml_client.spaces.list()

# Find and set the default space
space_name=PRE_PROD_SPACE_NAME
spaces = wml_client.spaces.get_details()['resources']
space_id = None
for space in spaces:
    if space['entity']['name'] == space_name:
        space_id = space["metadata"]["id"]
        print(space_id)
if space_id is None:
    space_id = wml_client.spaces.store(
        meta_props={wml_client.spaces.ConfigurationMetaNames.NAME: space_name})["metadata"]["id"]

wml_client.set.default_space(space_id)



/usr/local/lib/python3.10/site-packages/ibm_watson_machine_learning/client.py:184: UserWarning: The provided version: 4.7 is different from the current CP4D version: 5.0. Correct the wml_credentials with proper CP4D version number.
  warn(


https://cpd-cpd-instance.apps.wos415nfs2672.cp.fyre.ibm.com
Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  -----------------------------------------------------------------------  ------------------------
ID                                    NAME                                                                     CREATED
498630dc-3e0b-4470-b44d-583f2a393c0c  wml_preprod_space                                                        2024-08-08T00:19:18.705Z
00969846-29b6-446a-959e-587df21992e0  prod                                                                     2024-08-07T06:33:55.297Z
06873ce9-baf3-46a6-86c4-54c1e3d84429  tutorial_space                                                           2024-08-06T13:29:18.489Z
e8b87647-a0e9-4932-920f-1e8f1d1f383d  tutorial-space                                                           2024-08-06T12:55:07.962Z
832d6bbe-a2ea-451f-8d94-853da7c52b0

'SUCCESS'

In [12]:
wml_client.version

'1.0.360'

## Deploy the Spark Credit Risk Model to WML

The following cell deploys the Spark version of the German Credit Risk Model to the specified Machine Learning instance in the specified deployment space. You'll notice that this version of the German Credit Risk model has an auc-roc score around 71%.

In [13]:
import numpy 
numpy.version.version

import pandas as pd
import json

from pyspark import SparkContext, SQLContext
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier,GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import StringIndexer, VectorAssembler, IndexToString
from pyspark.sql.types import StructType, DoubleType, StringType, ArrayType

from pyspark.sql import SparkSession
from pyspark import SparkFiles

spark = SparkSession.builder.getOrCreate()
pd_data = pd.read_csv("german_credit_data_biased_training.csv", sep=",", header=0)
spark_df = spark.read.csv(path="german_credit_data_biased_training.csv", sep=",", header=True, inferSchema=True)
spark_df.head()

(train_data, test_data) = spark_df.randomSplit([0.9, 0.1], 24)
print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))

si_CheckingStatus = StringIndexer(inputCol='CheckingStatus', outputCol='CheckingStatus_IX')
si_CreditHistory = StringIndexer(inputCol='CreditHistory', outputCol='CreditHistory_IX')
si_LoanPurpose = StringIndexer(inputCol='LoanPurpose', outputCol='LoanPurpose_IX')
si_ExistingSavings = StringIndexer(inputCol='ExistingSavings', outputCol='ExistingSavings_IX')
si_EmploymentDuration = StringIndexer(inputCol='EmploymentDuration', outputCol='EmploymentDuration_IX')
si_Sex = StringIndexer(inputCol='Sex', outputCol='Sex_IX')
si_OthersOnLoan = StringIndexer(inputCol='OthersOnLoan', outputCol='OthersOnLoan_IX')
si_OwnsProperty = StringIndexer(inputCol='OwnsProperty', outputCol='OwnsProperty_IX')
si_InstallmentPlans = StringIndexer(inputCol='InstallmentPlans', outputCol='InstallmentPlans_IX')
si_Housing = StringIndexer(inputCol='Housing', outputCol='Housing_IX')
si_Job = StringIndexer(inputCol='Job', outputCol='Job_IX')
si_Telephone = StringIndexer(inputCol='Telephone', outputCol='Telephone_IX')
si_ForeignWorker = StringIndexer(inputCol='ForeignWorker', outputCol='ForeignWorker_IX')
si_Label = StringIndexer(inputCol="Risk", outputCol="label").fit(spark_df)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_Label.labels)

va_features = VectorAssembler(
inputCols=["CheckingStatus_IX", "CreditHistory_IX", "LoanPurpose_IX", "ExistingSavings_IX",
           "EmploymentDuration_IX", "Sex_IX", "OthersOnLoan_IX", "OwnsProperty_IX", "InstallmentPlans_IX",
           "Housing_IX", "Job_IX", "Telephone_IX", "ForeignWorker_IX", "LoanDuration", "LoanAmount",
           "InstallmentPercent", "CurrentResidenceDuration", "LoanDuration", "Age", "ExistingCreditsCount",
           "Dependents"], outputCol="features")

classifier=GBTClassifier(featuresCol="features")

pipeline = Pipeline(
stages=[si_CheckingStatus, si_CreditHistory, si_EmploymentDuration, si_ExistingSavings, si_ForeignWorker,
        si_Housing, si_InstallmentPlans, si_Job, si_LoanPurpose, si_OthersOnLoan,
        si_OwnsProperty, si_Sex, si_Telephone, si_Label, va_features, classifier, label_converter])

model = pipeline.fit(train_data)
predictions = model.transform(test_data)
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction")
auc = evaluator.evaluate(predictions)

print("Accuracy = %g" % auc)

# Remove existing model and deployment
MODEL_NAME=PRE_PROD_MODEL_NAME
DEPLOYMENT_NAME=PRE_PROD_DEPLOYMENT_NAME

deployment_details = wml_client.deployments.get_details()
# for deployment in deployment_details['resources']:
#     deployment_id = wml_client.deployments.get_id(deployment)
#     model_id = deployment['entity']['asset']['id']
#     if deployment['entity']['name'] == DEPLOYMENT_NAME:
#         print('Deleting deployment id', deployment_id)
#         wml_client.deployments.delete(deployment_id)
#         print('Deleting model id', model_id)
#         wml_client.repository.delete(model_id)
wml_client.repository.list_models()

# Save Model
software_spec_uid = wml_client.software_specifications.get_id_by_name("spark-mllib_3.3")
model_props_rf = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME,
    wml_client.repository.ModelMetaNames.DESCRIPTION: MODEL_NAME,
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
    wml_client.repository.ModelMetaNames.TYPE: 'mllib_3.3'
}
print(model_props_rf)
published_model_details = wml_client.repository.store_model(model=model, meta_props=model_props_rf, training_data=train_data, pipeline=pipeline)
print(published_model_details)

# List models in the repository
wml_client.repository.list_models()

# Get the model UID
pre_prod_model_uid = wml_client.repository.get_model_uid(published_model_details)
pre_prod_model_uid


# Deploy model
wml_deployments = wml_client.deployments.get_details()
pre_prod_deployment_uid = None
for deployment in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        pre_prod_deployment_uid = wml_client.deployments.get_id(deployment)
        break

if pre_prod_deployment_uid is None:
    print("Deploying model...")
    meta_props = {
        wml_client.deployments.ConfigurationMetaNames.NAME: DEPLOYMENT_NAME,
        wml_client.deployments.ConfigurationMetaNames.DESCRIPTION: DEPLOYMENT_NAME,
        wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
    deployment = wml_client.deployments.create(artifact_uid=pre_prod_model_uid, name=DEPLOYMENT_NAME, meta_props=meta_props)
    pre_prod_deployment_uid = wml_client.deployments.get_id(deployment)

print("Model id: {}".format(pre_prod_model_uid))
print("Deployment id: {}".format(pre_prod_deployment_uid))

pre_prod_deployment_uid=wml_client.deployments.get_id(deployment)
pre_prod_deployment_uid

fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
values = [
  ["no_checking",13,"credits_paid_to_date","car_new",1343,"100_to_500","1_to_4",2,"female","none",3,"savings_insurance",46,"none","own",2,"skilled",1,"none","yes"],
  ["no_checking",24,"prior_payments_delayed","furniture",4567,"500_to_1000","1_to_4",4,"male","none",4,"savings_insurance",36,"none","free",2,"management_self-employed",1,"none","yes"],
  ["0_to_200",26,"all_credits_paid_back","car_new",863,"less_100","less_1",2,"female","co-applicant",2,"real_estate",38,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",14,"no_credits","car_new",2368,"less_100","1_to_4",3,"female","none",3,"real_estate",29,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",4,"no_credits","car_new",250,"less_100","unemployed",2,"female","none",3,"real_estate",23,"none","rent",1,"management_self-employed",1,"none","yes"],
  ["no_checking",17,"credits_paid_to_date","car_new",832,"100_to_500","1_to_4",2,"male","none",2,"real_estate",42,"none","own",1,"skilled",1,"none","yes"],
  ["no_checking",33,"outstanding_credit","appliances",5696,"unknown","greater_7",4,"male","co-applicant",4,"unknown",54,"none","free",2,"skilled",1,"yes","yes"],
  ["0_to_200",13,"prior_payments_delayed","retraining",1375,"100_to_500","4_to_7",3,"male","none",3,"real_estate",37,"none","own",2,"management_self-employed",1,"none","yes"]
]

payload_scoring = {"fields": fields,"values": values}
payload = {
    wml_client.deployments.ScoringMetaNames.INPUT_DATA: [payload_scoring]
}
scoring_response = wml_client.deployments.score(pre_prod_deployment_uid, payload)

scoring_response

24/08/08 05:50:54 WARN Utils: Your hostname, Nelwins-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.103 instead (on interface en0)
24/08/08 05:50:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/08 05:50:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Number of records for training: 4485
Number of records for evaluation: 515


24/08/08 05:51:05 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/08/08 05:51:05 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


Accuracy = 0.726321


24/08/08 05:51:08 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


--  ----  -------  ----  ----------  ----------------
ID  NAME  CREATED  TYPE  SPEC_STATE  SPEC_REPLACEMENT
--  ----  -------  ----  ----------  ----------------
{'name': 'German Credit Risk Model - PreProd', 'description': 'German Credit Risk Model - PreProd', 'software_spec': 'd11f2434-4fc7-58b7-8a62-755da64fdaf8', 'type': 'mllib_3.3'}
Note: Warnings!! :  Software specification spark-mllib_3.3 specified for the wml_model is deprecated and will be removed in the future. We recommend you use spark-mllib_3.4 instead. For details see Supported Frameworks https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=specifications-supported-deployment-frameworks
{'entity': {'hybrid_pipeline_software_specs': [], 'label_column': 'Risk', 'pipeline': {'id': '6474bdbb-3344-48b0-b58b-664913fb21eb'}, 'schemas': {'input': [{'fields': [{'metadata': {}, 'name': 'CheckingStatus', 'nullable': True, 'type': 'string'}, {'metadata': {}, 'name': 'LoanDuration', 'nullable': True, 'type': 'integer'}, {'metadata

/usr/local/lib/python3.10/site-packages/ibm_watson_machine_learning/repository.py:563: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


Deploying model...


#######################################################################################

Synchronous deployment creation for uid: 'fe7ed7dd-2856-42d6-83ab-51c13079f78d' started

#######################################################################################


initializing
Note: Software specification spark-mllib_3.3 is deprecated. Use spark-mllib_3.4 software specification instead when saving a spark model. For details, see https://www.ibm.com/support/producthub/icpdata/docs/content/SSQNUZ_latest/wsj/wmls/wmls-deploy-python-types.html.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='42b79776-470c-4d05-9115-76c4f5fffeab'
------------------------------------------------------------------------------------------------


Model id: fe7ed7dd-2856-42d6-83ab-51c13079f78d
Deployment id: 42b79776-470c-4d05-9115-76c4f5fffeab


{'predictions': [{'fields': ['CheckingStatus',
    'LoanDuration',
    'CreditHistory',
    'LoanPurpose',
    'LoanAmount',
    'ExistingSavings',
    'EmploymentDuration',
    'InstallmentPercent',
    'Sex',
    'OthersOnLoan',
    'CurrentResidenceDuration',
    'OwnsProperty',
    'Age',
    'InstallmentPlans',
    'Housing',
    'ExistingCreditsCount',
    'Job',
    'Dependents',
    'Telephone',
    'ForeignWorker',
    'CheckingStatus_IX',
    'CreditHistory_IX',
    'EmploymentDuration_IX',
    'ExistingSavings_IX',
    'ForeignWorker_IX',
    'Housing_IX',
    'InstallmentPlans_IX',
    'Job_IX',
    'LoanPurpose_IX',
    'OthersOnLoan_IX',
    'OwnsProperty_IX',
    'Sex_IX',
    'Telephone_IX',
    'features',
    'rawPrediction',
    'probability',
    'prediction',
    'predictedLabel'],
   'values': [['no_checking',
     13,
     'credits_paid_to_date',
     'car_new',
     1343,
     '100_to_500',
     '1_to_4',
     2,
     'female',
     'none',
     3,
     'savings

## Deploy the Scikit-Learn Credit Risk Model to Watson Machine Learning

The following cell deploys the Scikit-learn version of the German Credit Risk Model to the specified Machine Learning instance in the specified deployment space. This version of the German Credit Risk model has an auc-roc score around 85% and will be called the "Challenger."

Note : If you are running this notebook on python 3.10, then use `runtime-23.1-py3.10` and `scikit-learn_1.1` as software specifications name and ModelMetaNames.TYPE respectively

In [14]:
import sklearn
sklearn.__version__

'1.1.1'

In [15]:
import pandas as pd
import json
import sys
import numpy
import sklearn
import sklearn.ensemble
numpy.set_printoptions(threshold=sys.maxsize)
#from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.utils.multiclass import type_of_target
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_validate
from sklearn.metrics import get_scorer
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report

data_df=pd.read_csv ("german_credit_data_biased_training.csv")

data_df.head()

target_label_name = "Risk"
feature_cols= data_df.drop(columns=[target_label_name])
label= data_df[target_label_name]

# Set model evaluation properties
optimization_metric = 'roc_auc'
random_state = 33
cv_num_folds = 3
holdout_fraction = 0.1

if type_of_target(label.values) in ['multiclass', 'binary']:
    X_train, X_holdout, y_train, y_holdout = train_test_split(feature_cols, label, test_size=holdout_fraction, random_state=random_state, stratify=label.values)
else:
    X_train, X_holdout, y_train, y_holdout = train_test_split(feature_cols, label, test_size=holdout_fraction, random_state=random_state)

# Data preprocessing transformer generation

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('OrdinalEncoder', OrdinalEncoder(categories='auto',dtype=numpy.float64 ))])

numeric_features = feature_cols.select_dtypes(include=['int64', 'float64']).columns
categorical_features = feature_cols.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Initiate model and create pipeline
model=GradientBoostingClassifier()
gbt_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])
model_gbt=gbt_pipeline.fit(X_train, y_train)

y_pred = model_gbt.predict(X_holdout)


# Evaluate model performance on test data and Cross validation
scorer = get_scorer(optimization_metric)
scorer(model_gbt,X_holdout, y_holdout)

# Cross validation -3 folds
cv_results = cross_validate(model_gbt,X_train,y_train, scoring={optimization_metric:scorer})
numpy.mean(cv_results['test_' + optimization_metric])

print(classification_report(y_pred, y_holdout))


# Remove existing model and deployment
MODEL_NAME=PRE_PROD_CHALLENGER_MODEL_NAME
DEPLOYMENT_NAME=PRE_PROD_CHALLENGER_DEPLOYMENT_NAME

deployment_details = wml_client.deployments.get_details()
for deployment in deployment_details['resources']:
    deployment_id = wml_client.deployments.get_id(deployment)
    model_id = deployment['entity']['asset']['id']
    if deployment['entity']['name'] == DEPLOYMENT_NAME:
        print('Deleting deployment id', deployment_id)
        wml_client.deployments.delete(deployment_id)
        print('Deleting model id', model_id)
        wml_client.repository.delete(model_id)
wml_client.repository.list_models()

# Store Model
software_spec_uid = wml_client.software_specifications.get_id_by_name("runtime-23.1-py3.10")
model_props_gbt = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME,
    wml_client.repository.ModelMetaNames.DESCRIPTION: MODEL_NAME,
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
    wml_client.repository.ModelMetaNames.TYPE: "scikit-learn_1.1"
}

published_model_details = wml_client.repository.store_model(model=model_gbt, meta_props=model_props_gbt, training_data=feature_cols,training_target=label)
print(published_model_details)

# List models in the repository
wml_client.repository.list_models()

# Get the model UID
challenger_model_uid = wml_client.repository.get_model_id(published_model_details)
print('challenger_model_uid ', challenger_model_uid)


# Deploy model
wml_deployments = wml_client.deployments.get_details()
challenger_deployment_uid = None
for deployment in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        challenger_deployment_uid = wml_client.deployments.get_id(deployment)
        break

if challenger_deployment_uid is None:
    print("Deploying model...")
    meta_props = {
        wml_client.deployments.ConfigurationMetaNames.NAME: DEPLOYMENT_NAME,
        wml_client.deployments.ConfigurationMetaNames.DESCRIPTION: DEPLOYMENT_NAME,
        wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
    deployment = wml_client.deployments.create(artifact_uid=challenger_model_uid, meta_props=meta_props)
    challenger_deployment_uid = wml_client.deployments.get_id(deployment)

print("Model id: {}".format(challenger_model_uid))
print("Deployment id: {}".format(challenger_deployment_uid))

challenger_deployment_uid=wml_client.deployments.get_id(deployment)
challenger_deployment_uid


# Sample scoring
fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
values = [
  ["no_checking",13,"credits_paid_to_date","car_new",1343,"100_to_500","1_to_4",2,"female","none",3,"savings_insurance",46,"none","own",2,"skilled",1,"none","yes"],
  ["no_checking",24,"prior_payments_delayed","furniture",4567,"500_to_1000","1_to_4",4,"male","none",4,"savings_insurance",36,"none","free",2,"management_self-employed",1,"none","yes"],
  ["0_to_200",26,"all_credits_paid_back","car_new",863,"less_100","less_1",2,"female","co-applicant",2,"real_estate",38,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",14,"no_credits","car_new",2368,"less_100","1_to_4",3,"female","none",3,"real_estate",29,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",4,"no_credits","car_new",250,"less_100","unemployed",2,"female","none",3,"real_estate",23,"none","rent",1,"management_self-employed",1,"none","yes"],
  ["no_checking",17,"credits_paid_to_date","car_new",832,"100_to_500","1_to_4",2,"male","none",2,"real_estate",42,"none","own",1,"skilled",1,"none","yes"],
  ["no_checking",33,"outstanding_credit","appliances",5696,"unknown","greater_7",4,"male","co-applicant",4,"unknown",54,"none","free",2,"skilled",1,"yes","yes"],
  ["0_to_200",13,"prior_payments_delayed","retraining",1375,"100_to_500","4_to_7",3,"male","none",3,"real_estate",37,"none","own",2,"management_self-employed",1,"none","yes"]
]

payload_scoring = {"fields": fields,"values": values}
payload = {
    wml_client.deployments.ScoringMetaNames.INPUT_DATA: [payload_scoring]
}
scoring_response = wml_client.deployments.score(challenger_deployment_uid, payload)

scoring_response


Number of records for training: 4485
Number of records for evaluation: 515
Accuracy = 0.726321
------------------------------------  -------------------------------------  ------------------------  ----------------  ----------  ----------------
ID                                    NAME                                   CREATED                   TYPE              SPEC_STATE  SPEC_REPLACEMENT
f94340ae-57bf-4e49-be5c-7156cd0ec09a  German Credit Risk Model - Challenger  2024-08-08T00:24:24.002Z  scikit-learn_1.1  supported
fe7ed7dd-2856-42d6-83ab-51c13079f78d  German Credit Risk Model - PreProd     2024-08-08T00:21:19.002Z  mllib_3.3         deprecated  spark-mllib_3.4
------------------------------------  -------------------------------------  ------------------------  ----------------  ----------  ----------------
{'name': 'German Credit Risk Model - PreProd', 'description': 'German Credit Risk Model - PreProd', 'software_spec': 'd11f2434-4fc7-58b7-8a62-755da64fdaf8', 'type': 'mllib_3.3

{'predictions': [{'fields': ['CheckingStatus',
    'LoanDuration',
    'CreditHistory',
    'LoanPurpose',
    'LoanAmount',
    'ExistingSavings',
    'EmploymentDuration',
    'InstallmentPercent',
    'Sex',
    'OthersOnLoan',
    'CurrentResidenceDuration',
    'OwnsProperty',
    'Age',
    'InstallmentPlans',
    'Housing',
    'ExistingCreditsCount',
    'Job',
    'Dependents',
    'Telephone',
    'ForeignWorker',
    'CheckingStatus_IX',
    'CreditHistory_IX',
    'EmploymentDuration_IX',
    'ExistingSavings_IX',
    'ForeignWorker_IX',
    'Housing_IX',
    'InstallmentPlans_IX',
    'Job_IX',
    'LoanPurpose_IX',
    'OthersOnLoan_IX',
    'OwnsProperty_IX',
    'Sex_IX',
    'Telephone_IX',
    'features',
    'rawPrediction',
    'probability',
    'prediction',
    'predictedLabel'],
   'values': [['no_checking',
     13,
     'credits_paid_to_date',
     'car_new',
     1343,
     '100_to_500',
     '1_to_4',
     2,
     'female',
     'none',
     3,
     'savings

# Configure OpenScale 
The notebook will now import the necessary libraries and set up a Python OpenScale client.

In [16]:
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator
from ibm_watson_openscale import APIClient
from ibm_watson_openscale.utils import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord
import uuid

In [17]:
authenticator = CloudPakForDataAuthenticator(
        url=WOS_CREDENTIALS['url'],
        username=WOS_CREDENTIALS['username'],
        password=WOS_CREDENTIALS['password'],
        disable_ssl_verification=True
    )

wos_client = APIClient(service_url=WOS_CREDENTIALS['url'],authenticator=authenticator)
wos_client.version

'3.0.39'

## Create schema and datamart

### Set up datamart
Watson OpenScale uses a database to store payload logs and calculated metrics. If an OpenScale datamart exists in Db2, the existing datamart will be used and no data will be overwritten.

In [18]:
wos_client.data_marts.show()

AIOSFASTPATHICP-00000000-0000-0000-0000-000000000000,Data Mart created by OpenScale ExpressPath,False,active,2024-06-04 05:19:03.698000+00:00,00000000-0000-0000-0000-000000000000


In [19]:
data_marts = wos_client.data_marts.list().result.data_marts
if len(data_marts) == 0:
    if DATABASE_CREDENTIALS is not None:
        if SCHEMA_NAME is None: 
            print("Please specify the SCHEMA_NAME and rerun the cell")

        print("Setting up external datamart")
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook",
                database_configuration=DatabaseConfigurationRequest(
                  database_type=DatabaseType.DB2,
                    credentials=PrimaryStorageCredentialsLong(
                        hostname=DATABASE_CREDENTIALS["hostname"],
                        username=DATABASE_CREDENTIALS["username"],
                        password=DATABASE_CREDENTIALS["password"],
                        db=DATABASE_CREDENTIALS["db"],
                        port=DATABASE_CREDENTIALS["port"]
                    ),
                    location=LocationSchemaName(
                        schema_name= SCHEMA_NAME
                    )
                )
             ).result
    else:
        print("Setting up internal datamart")
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook", 
                internal_database = True).result
        
    data_mart_id = added_data_mart_result.metadata.id
    
else:
    data_mart_id=data_marts[0].metadata.id
    print("Using existing datamart {}".format(data_mart_id))

Using existing datamart 00000000-0000-0000-0000-000000000000


In [20]:
data_mart_id

'00000000-0000-0000-0000-000000000000'

## Generate an ICP token

The following is a function that will generate an ICP access token used to interact with the Watson OpenScale APIs

In [21]:
import requests
import urllib3
from http import HTTPStatus

def get_iamtoken(url, username, password):
    fqdn = urllib3.util.parse_url(url).netloc
    domain = '.'.join(fqdn.split('.')[1:])
    token_url = 'https://cp-console.{}/idprovider/v1/auth/identitytoken'.format(domain)
    data = {
        'grant_type': 'password',
        'username': username,
        'password': password,
        'scope': 'openid'
    }
    return requests.post(token_url, data, verify=False)

def get_accesstoken(url, username, iamtoken):
    url = '{}/v1/preauth/validateAuth'.format(url)
    headers = {
        'Content-type': 'application/json',
        'username': username,
        'iam-token': iamtoken
    }
    return requests.get(url, headers=headers, verify=False)

def generate_access_token():
    url=WOS_CREDENTIALS['url']
    username=WOS_CREDENTIALS['username']
    password=WOS_CREDENTIALS['password'] 
    response = get_iamtoken(url,username,password)
    #service is not available when iamintegration=false so fall back to old way of generating code
    if response.status_code==HTTPStatus.SERVICE_UNAVAILABLE:
        url = '{}/v1/preauth/validateAuth'.format(url)
        headers = {'Content-type': 'application/json'}
        data = {
            'grant_type': 'password',
            'username': username,
            'password': password
        }
        return requests.get(url, headers=headers, auth= (username,password),verify=False).json()['accessToken']
        
    else:
        return get_accesstoken(url,username, response.json()['access_token']).json()['accessToken']

## Bind WML machine learning instance as Pre-Prod

Watson OpenScale needs to be bound to the Watson Machine Learning instance to capture payload data into and out of the model. If a binding with name "WML Pre-Prod" already exists, this code will delete that binding a create a new one.

**Note**: Binding with name `WML Pre-Prod` is assumed to be only created by this notebook.

In [22]:
SERVICE_PROVIDER_NAME = "WML Pre-Prod"
SERVICE_PROVIDER_DESCRIPTION = "Added by tutorial WOS notebook."

In [23]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

In [24]:
added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.WATSON_MACHINE_LEARNING,
        deployment_space_id = space_id,
        operational_space_id = "pre_production",
        credentials=WMLCredentialsCP4D(),
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id




 Waiting for end of adding service provider 33397d85-dcae-4703-907d-4668932f7df0 




active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------




In [25]:
wos_client.service_providers.show()

99999999-9999-9999-9999-999999999999,active,WML Pre-Prod-2,watson_machine_learning,2024-08-08 00:28:02.977000+00:00,33397d85-dcae-4703-907d-4668932f7df0
99999999-9999-9999-9999-999999999999,active,Watson Machine Learning V2-2,watson_machine_learning,2024-08-07 09:57:15.265000+00:00,cf31400f-925c-4180-b4b2-77e53440d9a7
99999999-9999-9999-9999-999999999999,active,WML - for training data -2,watson_machine_learning,2024-08-07 08:42:56.271000+00:00,1b7f6643-f6ac-4754-9f39-e8dbe5b09232
99999999-9999-9999-9999-999999999999,active,Watson Machine Learning V2_test-2,watson_machine_learning,2024-08-07 06:51:26.963000+00:00,b9fd3cb5-c7a4-4b5c-a90c-684676276352
99999999-9999-9999-9999-999999999999,active,WML Prod,watson_machine_learning,2024-08-07 06:34:58.909000+00:00,f4931b54-2103-40d9-b564-9ba3b1982fdb
99999999-9999-9999-9999-999999999999,active,WML Pre-Prod2,watson_machine_learning,2024-08-07 06:03:11.179000+00:00,81655d73-6ea3-44ae-9876-86df1fa6390d
99999999-9999-9999-9999-999999999999,active,WML AI function - WOS notebook,watson_machine_learning,2024-08-06 13:32:38.884000+00:00,e413a76d-802c-44f3-8c55-300c5aa2659b
99999999-9999-9999-9999-999999999999,active,Watson Machine Learning - Indirect Bias Demo-2,watson_machine_learning,2024-08-06 13:25:37.046000+00:00,c3601bc6-22c9-45a1-b016-c29cf0bf64b4
None,active,IAE7,custom_machine_learning,2024-07-02 09:08:06.273000+00:00,184e73a2-7fd8-4f3f-b994-bb648f6eb8ec
None,active,IAE6,custom_machine_learning,2024-07-02 09:04:22.643000+00:00,644befcd-6d36-4f4d-a30a-cd51a28b63fe


Note: First 10 records were displayed.


## Create an integration to IBM OpenPages

To push metrics from Watson OpenScale to IBM OpenPages, a connection must be established between the two services.

In [26]:
headers = {}
headers["Content-Type"] = "application/json"
headers["Authorization"] = "Bearer {}".format(generate_access_token())

INTEGRATED_SYSTEMS_URL = WOS_CREDENTIALS["url"] + "/openscale/{0}/v2/integrated_systems".format(WOS_GUID)

payload = {
    "name": "OpenPages Connection",
    "type": "open_pages",
    "description": "Integration with OpenPages",
    "credentials": OPENPAGES_CREDENTIALS
}

response = requests.post(INTEGRATED_SYSTEMS_URL, json=payload, headers=headers, verify=False)
json_data = response.json()
print(json_data)

if "metadata" in json_data and "id" in json_data["metadata"]:
    integrated_system_id = json_data["metadata"]["id"]
    print(integrated_system_id)

{'entity': {'credentials': {'secret_id': '2e612f85-588a-4e14-ac61-1f0be9db9945'}, 'description': 'Integration with OpenPages', 'name': 'OpenPages Connection', 'status': {'state': 'active'}, 'type': 'open_pages'}, 'metadata': {'created_at': '2024-08-08T00:28:12.866Z', 'created_by': 'cpadmin', 'crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:integrated_system:cf7e2e39-78c6-4711-8d76-89fa08acc9a0', 'id': 'cf7e2e39-78c6-4711-8d76-89fa08acc9a0', 'url': '/v2/integrated_systems/cf7e2e39-78c6-4711-8d76-89fa08acc9a0'}}
cf7e2e39-78c6-4711-8d76-89fa08acc9a0


## Subscriptions
### Remove existing PreProd and Challenger credit risk subscriptions
This code removes previous subscriptions with name `German Credit Risk Model - PreProd` and `German Credit Risk Model - Challenger` to refresh the monitors with the new model and new data.

In [27]:
subscriptions = wos_client.subscriptions.list().result.subscriptions
for subscription in subscriptions:
    sub_name = subscription.entity.asset.name
    if sub_name == PRE_PROD_MODEL_NAME or sub_name == PRE_PROD_CHALLENGER_MODEL_NAME:
        wos_client.subscriptions.delete(subscription.metadata.id)
        print('Deleted existing subscription for', sub_name)

Deleted existing subscription for German Credit Risk Model - Challenger
Deleted existing subscription for German Credit Risk Model - PreProd


In [28]:
wos_client.subscriptions.show()

f1b4fe35-1a75-4dca-a9b3-bc5dd7ed43db,model,German Credit Risk Model - PreProd,00000000-0000-0000-0000-000000000000,42b13344-baa1-45a0-89ca-ef662387f691,German Credit Risk Model - PreProd,81655d73-6ea3-44ae-9876-86df1fa6390d,pending_delete,2024-08-07 06:07:24.153000+00:00,61003e50-20d6-4ce8-a379-013a03c44229
67e11064-81b2-4ea8-addf-b39c218299b4,model,German Credit Risk Model - Challenger,00000000-0000-0000-0000-000000000000,978e26e1-509c-4e03-b20a-baacfbbe222b,German Credit Risk Model - Challenger,81655d73-6ea3-44ae-9876-86df1fa6390d,pending_delete,2024-08-07 06:07:38.473000+00:00,c0174a6c-d8e6-458b-8549-cd03d7df486e
1b9f22c5-ac03-4017-b40b-27969e429379,model,Scikit German Risk Model WML V4 - zLinux,00000000-0000-0000-0000-000000000000,edc1e06c-7bf9-4522-8893-05a919a35ca0,Scikit German Risk Deployment WML V4 - zLinux,cf31400f-925c-4180-b4b2-77e53440d9a7,active,2024-08-07 09:57:28.433000+00:00,ad19393e-2078-4316-ac19-7de0ddfd7937
df904b8a-2884-4318-82df-35cb430f8587,model,Scikit German Risk Model - for training data - zLinux,00000000-0000-0000-0000-000000000000,17e4097d-c4b5-41fe-bba5-3e12af95041e,Scikit German Risk Model - for training data - zLinux,1b7f6643-f6ac-4754-9f39-e8dbe5b09232,active,2024-08-07 08:51:02.441000+00:00,bcbbcc1f-7e1c-4746-8055-128ce8d805ff
20516963-37ff-4769-a41f-b509827f3076,model,Text Binary Classifier,00000000-0000-0000-0000-000000000000,18c8f8d4-cfbf-4fbb-a51a-5ac59d2e9e20,Text Binary Classifier deployment,b9fd3cb5-c7a4-4b5c-a90c-684676276352,active,2024-08-07 06:51:41.900000+00:00,d673e127-dd69-4b05-a204-5b90332d7e72
a4a1a71d-2a44-43f8-b92a-ba3c80dc36fd,model,Credit Risk python Fn Model,00000000-0000-0000-0000-000000000000,90b19695-de03-41b9-b820-2d3e11c2f662,dep_Credit Risk python Fn Model,e413a76d-802c-44f3-8c55-300c5aa2659b,active,2024-08-06 13:32:53.985000+00:00,77c99979-8408-40b7-ab3a-e3c0a8a80aab
eed8b225-19e5-4460-9b6f-7271dc1e3ff2,model,Adult Census Income Classifier Model,00000000-0000-0000-0000-000000000000,038b8c68-6615-4c0d-8d46-15783d6ab9c8,Adult Census Income Classifier Deployment,c3601bc6-22c9-45a1-b016-c29cf0bf64b4,active,2024-08-06 13:26:09.549000+00:00,c274e76c-6bb9-43ce-ba64-127a3db95ede
438ca544-9bd1-48c2-8e8d-3de4ef4ca79b,model,WML_IAE4,00000000-0000-0000-0000-000000000000,78a0af9e-1014-4fb1-b22a-5e11f4fd70e7,WML_IAE4,d90c6bf2-49c6-4179-9876-8b85b0247d95,active,2024-07-02 07:03:31.504000+00:00,e34b9b87-b6e1-4c53-b92e-cb80dea042be
592b902d-3dc9-4e56-8bcb-86cbf1a6d8a9,model,gcr - P2 XGB Classifier - Model,00000000-0000-0000-0000-000000000000,2b976af0-e4ab-4859-af7d-2f2287d864ad,gcr model,4d2f2fb2-6b64-4d58-8f13-257166e468e9,active,2024-07-17 07:11:44.727000+00:00,e2df4ec7-6c75-416f-a444-8d21389f7513
e3ac9fc3-bccf-4a4e-b37b-490bfb93dd81,model,GCR AutoAI - P2 XGB Classifier - Model,00000000-0000-0000-0000-000000000000,6399e6e8-df5a-4370-9af4-34b2f2e76bc6,GCR Auto AI,a7ca157a-de07-457a-8c4c-b1a2e998699c,active,2024-07-03 10:37:20.487000+00:00,ce36911c-75f6-4f99-b4d2-b71e5d55a802


Note: First 10 records were displayed.


In [29]:
# Create preprod subscription
pre_prod_deployment = wml_client.deployments.get_details(deployment_uid=pre_prod_deployment_uid)
pre_prod_asset_details=wos_client.service_providers.get_deployment_asset(data_mart_id=data_mart_id,service_provider_id=service_provider_id,deployment_id=pre_prod_deployment_uid,deployment_space_id=space_id)
pre_prod_asset = Asset(
    asset_id=pre_prod_asset_details["entity"]["asset"]["asset_id"],
    name=pre_prod_asset_details["entity"]["asset"]["name"],
    url=pre_prod_asset_details["entity"]["asset"]["url"],
    asset_type=AssetTypes.MODEL,
    input_data_type=InputDataType.STRUCTURED,
    problem_type=ProblemType.BINARY_CLASSIFICATION,
    model_type='mllib_3.3'
)
pre_prod_asset_deployment = AssetDeploymentRequest(
    deployment_id=pre_prod_deployment_uid,
    name=PRE_PROD_DEPLOYMENT_NAME,
    deployment_type=DeploymentTypes.ONLINE,
    url=pre_prod_deployment["entity"]["status"]["online_url"]["url"]
)

pre_prod_asset_properties_request = AssetPropertiesRequest(
    label_column="Risk",
    probability_fields=["probability"],
    prediction_field="predictedLabel",
    feature_fields=["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"],
    categorical_fields=["CheckingStatus","CreditHistory","LoanPurpose","ExistingSavings","EmploymentDuration","Sex","OthersOnLoan","OwnsProperty","InstallmentPlans","Housing","Job","Telephone","ForeignWorker"]
)

Note: Software specification spark-mllib_3.3 is deprecated. Use spark-mllib_3.4 software specification instead when saving a spark model. For details, see https://www.ibm.com/support/producthub/icpdata/docs/content/SSQNUZ_latest/wsj/wmls/wmls-deploy-python-types.html.


In [30]:
pre_prod_asset_details

{'metadata': {'guid': '42b79776-470c-4d05-9115-76c4f5fffeab',
  'created_at': '2024-08-08T00:21:34.088Z',
  'modified_at': '2024-08-08T00:21:34.088Z'},
 'entity': {'name': 'German Credit Risk Model - PreProd',
  'type': 'online',
  'description': 'German Credit Risk Model - PreProd',
  'scoring_endpoint': {'url': 'https://internal-nginx-svc:12443/ml/v4/deployments/42b79776-470c-4d05-9115-76c4f5fffeab/predictions'},
  'asset': {'asset_id': 'fe7ed7dd-2856-42d6-83ab-51c13079f78d',
   'url': 'https://internal-nginx-svc:12443/ml/v4/models/fe7ed7dd-2856-42d6-83ab-51c13079f78d?space_id=498630dc-3e0b-4470-b44d-583f2a393c0c&version=2020-06-12',
   'name': 'German Credit Risk Model - PreProd',
   'asset_type': 'model',
   'created_at': '2024-08-08T00:21:19.987Z',
   'modified_at': '2024-08-08T00:21:28.415Z'},
  'asset_properties': {'model_type': 'mllib_3.3',
   'runtime_environment': 'spark-3.3.0',
   'label_column': 'Risk',
   'input_data_schema': {'type': 'struct',
    'id': '1',
    'fields':

In [31]:
pre_prod_subscription = wos_client.subscriptions.add(
        data_mart_id=data_mart_id,
        service_provider_id=service_provider_id,
        asset=pre_prod_asset,
        deployment=pre_prod_asset_deployment,
        asset_properties=pre_prod_asset_properties_request).result
pre_prod_subscription_id = pre_prod_subscription.metadata.id
print(pre_prod_subscription_id)

06b4e163-86b5-493d-8e28-8606fb296dca


In [32]:
# Create challenger subscription
challenger_deployment = wml_client.deployments.get_details(deployment_uid=challenger_deployment_uid)
challenger_asset_details=wos_client.service_providers.get_deployment_asset(data_mart_id=data_mart_id,service_provider_id=service_provider_id,deployment_id=challenger_deployment_uid,deployment_space_id=space_id)
challenger_asset = Asset(
    asset_id=challenger_asset_details["entity"]["asset"]["asset_id"],
    name=challenger_asset_details["entity"]["asset"]["name"],
    url=challenger_asset_details["entity"]["asset"]["url"],
    asset_type=AssetTypes.MODEL,
    input_data_type=InputDataType.STRUCTURED,
    problem_type=ProblemType.BINARY_CLASSIFICATION,
    model_type='scikit-learn_0.23'
)
challenger_asset_deployment = AssetDeploymentRequest(
    deployment_id=challenger_deployment_uid,
    name=PRE_PROD_CHALLENGER_DEPLOYMENT_NAME,
    deployment_type=DeploymentTypes.ONLINE,
    url=challenger_deployment["entity"]["status"]["online_url"]["url"]
)

challenger_asset_properties_request = AssetPropertiesRequest(
    label_column="Risk",
    probability_fields=["probability"],
    prediction_field="prediction",
    feature_fields=["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"],
    categorical_fields=["CheckingStatus","CreditHistory","LoanPurpose","ExistingSavings","EmploymentDuration","Sex","OthersOnLoan","OwnsProperty","InstallmentPlans","Housing","Job","Telephone","ForeignWorker"]
)

Note: Software specification spark-mllib_3.3 is deprecated. Use spark-mllib_3.4 software specification instead when saving a spark model. For details, see https://www.ibm.com/support/producthub/icpdata/docs/content/SSQNUZ_latest/wsj/wmls/wmls-deploy-python-types.html.


In [33]:
challenger_subscription = wos_client.subscriptions.add(
        data_mart_id=data_mart_id,
        service_provider_id=service_provider_id,
        asset=challenger_asset,
        deployment=challenger_asset_deployment,
        asset_properties=challenger_asset_properties_request).result
challenger_subscription_id = challenger_subscription.metadata.id
print(challenger_subscription_id)

a9a2e1e9-181e-45bd-9fec-18eea1ddd3e7


In [34]:
wos_client.subscriptions.show()

fe7ed7dd-2856-42d6-83ab-51c13079f78d,model,German Credit Risk Model - PreProd,00000000-0000-0000-0000-000000000000,42b79776-470c-4d05-9115-76c4f5fffeab,German Credit Risk Model - Challenger,33397d85-dcae-4703-907d-4668932f7df0,active,2024-08-08 00:28:30.628000+00:00,a9a2e1e9-181e-45bd-9fec-18eea1ddd3e7
fe7ed7dd-2856-42d6-83ab-51c13079f78d,model,German Credit Risk Model - PreProd,00000000-0000-0000-0000-000000000000,42b79776-470c-4d05-9115-76c4f5fffeab,German Credit Risk Model - PreProd,33397d85-dcae-4703-907d-4668932f7df0,active,2024-08-08 00:28:25.288000+00:00,06b4e163-86b5-493d-8e28-8606fb296dca
1b9f22c5-ac03-4017-b40b-27969e429379,model,Scikit German Risk Model WML V4 - zLinux,00000000-0000-0000-0000-000000000000,edc1e06c-7bf9-4522-8893-05a919a35ca0,Scikit German Risk Deployment WML V4 - zLinux,cf31400f-925c-4180-b4b2-77e53440d9a7,active,2024-08-07 09:57:28.433000+00:00,ad19393e-2078-4316-ac19-7de0ddfd7937
df904b8a-2884-4318-82df-35cb430f8587,model,Scikit German Risk Model - for training data - zLinux,00000000-0000-0000-0000-000000000000,17e4097d-c4b5-41fe-bba5-3e12af95041e,Scikit German Risk Model - for training data - zLinux,1b7f6643-f6ac-4754-9f39-e8dbe5b09232,active,2024-08-07 08:51:02.441000+00:00,bcbbcc1f-7e1c-4746-8055-128ce8d805ff
20516963-37ff-4769-a41f-b509827f3076,model,Text Binary Classifier,00000000-0000-0000-0000-000000000000,18c8f8d4-cfbf-4fbb-a51a-5ac59d2e9e20,Text Binary Classifier deployment,b9fd3cb5-c7a4-4b5c-a90c-684676276352,active,2024-08-07 06:51:41.900000+00:00,d673e127-dd69-4b05-a204-5b90332d7e72
a4a1a71d-2a44-43f8-b92a-ba3c80dc36fd,model,Credit Risk python Fn Model,00000000-0000-0000-0000-000000000000,90b19695-de03-41b9-b820-2d3e11c2f662,dep_Credit Risk python Fn Model,e413a76d-802c-44f3-8c55-300c5aa2659b,active,2024-08-06 13:32:53.985000+00:00,77c99979-8408-40b7-ab3a-e3c0a8a80aab
eed8b225-19e5-4460-9b6f-7271dc1e3ff2,model,Adult Census Income Classifier Model,00000000-0000-0000-0000-000000000000,038b8c68-6615-4c0d-8d46-15783d6ab9c8,Adult Census Income Classifier Deployment,c3601bc6-22c9-45a1-b016-c29cf0bf64b4,active,2024-08-06 13:26:09.549000+00:00,c274e76c-6bb9-43ce-ba64-127a3db95ede
438ca544-9bd1-48c2-8e8d-3de4ef4ca79b,model,WML_IAE4,00000000-0000-0000-0000-000000000000,78a0af9e-1014-4fb1-b22a-5e11f4fd70e7,WML_IAE4,d90c6bf2-49c6-4179-9876-8b85b0247d95,active,2024-07-02 07:03:31.504000+00:00,e34b9b87-b6e1-4c53-b92e-cb80dea042be
592b902d-3dc9-4e56-8bcb-86cbf1a6d8a9,model,gcr - P2 XGB Classifier - Model,00000000-0000-0000-0000-000000000000,2b976af0-e4ab-4859-af7d-2f2287d864ad,gcr model,4d2f2fb2-6b64-4d58-8f13-257166e468e9,active,2024-07-17 07:11:44.727000+00:00,e2df4ec7-6c75-416f-a444-8d21389f7513
e3ac9fc3-bccf-4a4e-b37b-490bfb93dd81,model,GCR AutoAI - P2 XGB Classifier - Model,00000000-0000-0000-0000-000000000000,6399e6e8-df5a-4370-9af4-34b2f2e76bc6,GCR Auto AI,a7ca157a-de07-457a-8c4c-b1a2e998699c,active,2024-07-03 10:37:20.487000+00:00,ce36911c-75f6-4f99-b4d2-b71e5d55a802


Note: First 10 records were displayed.


In [35]:
pre_prod_subscription_id

'06b4e163-86b5-493d-8e28-8606fb296dca'

In [36]:
challenger_subscription_id

'a9a2e1e9-181e-45bd-9fec-18eea1ddd3e7'

## Patch the training data reference in both the preprod & challenger subscription

In [37]:
headers = {}
headers["Content-Type"] = "application/json"
headers["Authorization"] = "Bearer {}".format(generate_access_token())

training_data_reference = {
  "connection": {
    "connection_string": "jdbc:db2://dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net:50000/BLUDB:retrieveMessagesFromServerOnGetMessage=true;",
    "database_name": "BLUDB",
    "hostname": "dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net",
    "password": "khhz72v+6mcwwkfv",
    "username": "cmb91569"
  },
  "location": {
    "schema_name": "CMB91569",
    "table_name": "CREDIT_RISK_TRAIN_DATA"
  },
  "name": "German credit risk training data",
  "type": "db2"
}

payload = [
 {
   "op": "replace",
   "path": "/asset_properties/training_data_reference",
   "value": training_data_reference
 }
]

In [38]:
response = wos_client.subscriptions.update(pre_prod_subscription_id, payload)
response.result.to_dict()

{'metadata': {'id': '06b4e163-86b5-493d-8e28-8606fb296dca',
  'crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:subscription:06b4e163-86b5-493d-8e28-8606fb296dca',
  'url': '/v2/subscriptions/06b4e163-86b5-493d-8e28-8606fb296dca',
  'created_at': '2024-08-08T00:28:25.288000Z',
  'created_by': 'cpadmin',
  'modified_at': '2024-08-08T01:17:25.916000Z',
  'modified_by': 'cpadmin'},
 'entity': {'data_mart_id': '00000000-0000-0000-0000-000000000000',
  'service_provider_id': '33397d85-dcae-4703-907d-4668932f7df0',
  'asset': {'asset_id': 'fe7ed7dd-2856-42d6-83ab-51c13079f78d',
   'url': 'https://internal-nginx-svc:12443/ml/v4/models/fe7ed7dd-2856-42d6-83ab-51c13079f78d?space_id=498630dc-3e0b-4470-b44d-583f2a393c0c&version=2020-06-12',
   'name': 'German Credit Risk Model - PreProd',
   'asset_type': 'model',
   'problem_type': 'binary',
   'model_type': 'mllib_3.3',
   'input_data_type': 'structured'},
  'asset_properties': {'training_data_referenc

In [39]:
response = wos_client.subscriptions.update(challenger_subscription_id, payload)
response.result.to_dict()

{'metadata': {'id': 'a9a2e1e9-181e-45bd-9fec-18eea1ddd3e7',
  'crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:subscription:a9a2e1e9-181e-45bd-9fec-18eea1ddd3e7',
  'url': '/v2/subscriptions/a9a2e1e9-181e-45bd-9fec-18eea1ddd3e7',
  'created_at': '2024-08-08T00:28:30.628000Z',
  'created_by': 'cpadmin',
  'modified_at': '2024-08-08T01:17:29.858000Z',
  'modified_by': 'cpadmin'},
 'entity': {'data_mart_id': '00000000-0000-0000-0000-000000000000',
  'service_provider_id': '33397d85-dcae-4703-907d-4668932f7df0',
  'asset': {'asset_id': 'fe7ed7dd-2856-42d6-83ab-51c13079f78d',
   'url': 'https://internal-nginx-svc:12443/ml/v4/models/fe7ed7dd-2856-42d6-83ab-51c13079f78d?space_id=498630dc-3e0b-4470-b44d-583f2a393c0c&version=2020-06-12',
   'name': 'German Credit Risk Model - PreProd',
   'asset_type': 'model',
   'problem_type': 'binary',
   'model_type': 'scikit-learn_0.23',
   'input_data_type': 'structured'},
  'asset_properties': {'training_data_

### Score the model so we can configure monitors
Now that the WML service has been bound and the subscription has been created, we need to send a request to the model before we configure OpenScale. This allows OpenScale to create a payload log in the datamart with the correct schema, so it can capture data coming into and out of the model. First, the code gets the model deployment's endpoint URL, and then sends a few records for predictions.

In [40]:
fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
values = [
  ["no_checking",13,"credits_paid_to_date","car_new",1343,"100_to_500","1_to_4",2,"female","none",3,"savings_insurance",46,"none","own",2,"skilled",1,"none","yes"],
  ["no_checking",24,"prior_payments_delayed","furniture",4567,"500_to_1000","1_to_4",4,"male","none",4,"savings_insurance",36,"none","free",2,"management_self-employed",1,"none","yes"],
  ["0_to_200",26,"all_credits_paid_back","car_new",863,"less_100","less_1",2,"female","co-applicant",2,"real_estate",38,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",14,"no_credits","car_new",2368,"less_100","1_to_4",3,"female","none",3,"real_estate",29,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",4,"no_credits","car_new",250,"less_100","unemployed",2,"female","none",3,"real_estate",23,"none","rent",1,"management_self-employed",1,"none","yes"],
  ["no_checking",17,"credits_paid_to_date","car_new",832,"100_to_500","1_to_4",2,"male","none",2,"real_estate",42,"none","own",1,"skilled",1,"none","yes"],
  ["no_checking",33,"outstanding_credit","appliances",5696,"unknown","greater_7",4,"male","co-applicant",4,"unknown",54,"none","free",2,"skilled",1,"yes","yes"],
  ["0_to_200",13,"prior_payments_delayed","retraining",1375,"100_to_500","4_to_7",3,"male","none",3,"real_estate",37,"none","own",2,"management_self-employed",1,"none","yes"]
]

payload_scoring = {"fields": fields,"values": values}

In [42]:
payload = {
    wml_client.deployments.ScoringMetaNames.INPUT_DATA: [payload_scoring]
}
scoring_response = wml_client.deployments.score(pre_prod_deployment_uid, payload)

print('Single record Scoring result:', '\n fields:', scoring_response['predictions'][0]['fields'], '\n values: ', scoring_response['predictions'][0]['values'][0])

Single record Scoring result: 
 fields: ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', 'CheckingStatus_IX', 'CreditHistory_IX', 'EmploymentDuration_IX', 'ExistingSavings_IX', 'ForeignWorker_IX', 'Housing_IX', 'InstallmentPlans_IX', 'Job_IX', 'LoanPurpose_IX', 'OthersOnLoan_IX', 'OwnsProperty_IX', 'Sex_IX', 'Telephone_IX', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'] 
 values:  ['no_checking', 13, 'credits_paid_to_date', 'car_new', 1343, '100_to_500', '1_to_4', 2, 'female', 'none', 3, 'savings_insurance', 46, 'none', 'own', 2, 'skilled', 1, 'none', 'yes', 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, [21, [1, 3, 5, 13, 14, 15, 16, 17, 18, 19, 20], [1.0, 1.0, 1.0, 13

In [43]:
time.sleep(10)
pre_prod_pl_data_set_id = None
pre_prod_pl_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=pre_prod_subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if pre_prod_pl_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id: ", pre_prod_pl_data_set_id)
    pl_records_count = wos_client.data_sets.get_records_count(pre_prod_pl_data_set_id)
    print("Number of records in the payload logging table: {}".format(pl_records_count))
    if pl_records_count == 0:
        print("Automatic payload logging did not happen, performing explicit payload logging.")
        wos_client.data_sets.store_records(data_set_id=pre_prod_pl_data_set_id, request_body=[PayloadRecord(
                       scoring_id="mrm_"+str(uuid.uuid4()),
                       request=payload_scoring,
                       response=scoring_response['predictions'][0]
                   )])
        time.sleep(5)
        pl_records_count = wos_client.data_sets.get_records_count(pre_prod_pl_data_set_id)
        print("Number of records in the payload logging table: {}".format(pl_records_count))

Payload data set id:  037c48fa-9023-4544-abe3-bcf6e8f83b00
Number of records in the payload logging table: 8


In [44]:
payload = {
    wml_client.deployments.ScoringMetaNames.INPUT_DATA: [payload_scoring]
}
scoring_response = wml_client.deployments.score(challenger_deployment_uid, payload)

print('Single record Scoring result:', '\n fields:', scoring_response['predictions'][0]['fields'], '\n values: ', scoring_response['predictions'][0]['values'][0])

Single record Scoring result: 
 fields: ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', 'CheckingStatus_IX', 'CreditHistory_IX', 'EmploymentDuration_IX', 'ExistingSavings_IX', 'ForeignWorker_IX', 'Housing_IX', 'InstallmentPlans_IX', 'Job_IX', 'LoanPurpose_IX', 'OthersOnLoan_IX', 'OwnsProperty_IX', 'Sex_IX', 'Telephone_IX', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'] 
 values:  ['no_checking', 13, 'credits_paid_to_date', 'car_new', 1343, '100_to_500', '1_to_4', 2, 'female', 'none', 3, 'savings_insurance', 46, 'none', 'own', 2, 'skilled', 1, 'none', 'yes', 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, [21, [1, 3, 5, 13, 14, 15, 16, 17, 18, 19, 20], [1.0, 1.0, 1.0, 13

In [45]:
from ibm_watson_openscale.supporting_classes import *
time.sleep(10)
challenger_pl_data_set_id = None
challenger_pl_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=challenger_subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if challenger_pl_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id: ", challenger_pl_data_set_id)
    pl_records_count = wos_client.data_sets.get_records_count(challenger_pl_data_set_id)
    print("Number of records in the payload logging table: {}".format(pl_records_count))
    if pl_records_count == 0:
        print("Automatic payload logging did not happen, performing explicit payload logging.")
        wos_client.data_sets.store_records(data_set_id=challenger_pl_data_set_id, request_body=[PayloadRecord(
                       scoring_id="mrm_"+str(uuid.uuid4()),
                       request=payload_scoring,
                       response=scoring_response['predictions'][0]
                   )])
        time.sleep(5)
        pl_records_count = wos_client.data_sets.get_records_count(challenger_pl_data_set_id)
        print("Number of records in the payload logging table: {}".format(pl_records_count))

Payload data set id:  2f7ba47c-cff2-4897-a27e-55b8d60eaafa
Number of records in the payload logging table: 16


# Quality monitoring

## Enable quality monitoring
The code below waits ten seconds to allow the payload logging table to be set up before it begins enabling monitors. First, it turns on the quality (accuracy) monitor and sets an alert threshold of 80%. OpenScale will show an alert on the dashboard if the model accuracy measurement (area under the curve, in the case of a binary classifier) falls below this threshold.

The second paramater supplied, min_records, specifies the minimum number of feedback records OpenScale needs before it calculates a new measurement. The quality monitor runs hourly, but the accuracy reading in the dashboard will not change until an additional 50 feedback records have been added, via the user interface, the Python client, or the supplied feedback endpoint.

In [46]:
time.sleep(10)
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=pre_prod_subscription_id
)
parameters = {
    "min_feedback_data_size": 100,
    "threshold": 0.8
}
pre_prod_quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters
).result




 Waiting for end of monitor instance creation 0392d9f8-d375-49ef-b754-e3c6584070c7 




preparing
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [47]:
time.sleep(10)
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=challenger_subscription_id
)
parameters = {
    "min_feedback_data_size": 100,
    "threshold": 0.8
}
challenger_quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters
).result




 Waiting for end of monitor instance creation c0c3274d-dc61-4384-84e3-5d4b3b258379 




preparing
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




# Fairness, drift monitoring and explanations 

## Fairness configuration
The code below configures fairness monitoring for our model. It turns on monitoring for two features, Sex and Age. In each case, we must specify:

Which model feature to monitor
One or more majority groups, which are values of that feature that we expect to receive a higher percentage of favorable outcomes
One or more minority groups, which are values of that feature that we expect to receive a higher percentage of unfavorable outcomes
The threshold at which we would like OpenScale to display an alert if the fairness measurement falls below (in this case, 80%)
Additionally, we must specify which outcomes from the model are favourable outcomes, and which are unfavourable. We must also provide the number of records OpenScale will use to calculate the fairness score. In this case, OpenScale's fairness monitor will run hourly, but will not calculate a new fairness rating until at least 100 records have been added. Finally, to calculate fairness, OpenScale must perform some calculations on the training data, so we provide the dataframe containing the data.

In [48]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=pre_prod_subscription_id

)
parameters = {
    "features": [
        {"feature": "Sex",
         "majority": ['male'],
         "minority": ['female']
         },
        {"feature": "Age",
         "majority": [[26, 75]],
         "minority": [[18, 25]]
         }
    ],
    "favourable_class": ["No Risk"],
    "unfavourable_class": ["Risk"],
    "min_records": 100
}
thresholds = [{
    "metric_id": "fairness_value",
    "specific_values": [{
            "applies_to": [{
                "key": "feature",
                "type": "tag",
                "value": "Age"
            }],
            "value": 95
        },
        {
            "applies_to": [{
                "key": "feature",
                "type": "tag",
                "value": "Sex"
            }],
            "value": 95
        }
    ],
    "type": "lower_limit",
    "value": 80.0
}]

pre_prod_fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds).result
pre_prod_fairness_monitor_details.metadata.id




 Waiting for end of monitor instance creation 0654edac-ba0f-4167-87d6-9875b22e773c 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




'0654edac-ba0f-4167-87d6-9875b22e773c'

In [49]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=challenger_subscription_id

)
parameters = {
    "features": [
        {"feature": "Sex",
         "majority": ['male'],
         "minority": ['female']
         },
        {"feature": "Age",
         "majority": [[26, 75]],
         "minority": [[18, 25]]
         }
    ],
    "favourable_class": ["No Risk"],
    "unfavourable_class": ["Risk"],
    "min_records": 100
}
thresholds = [{
    "metric_id": "fairness_value",
    "specific_values": [{
            "applies_to": [{
                "key": "feature",
                "type": "tag",
                "value": "Age"
            }],
            "value": 95
        },
        {
            "applies_to": [{
                "key": "feature",
                "type": "tag",
                "value": "Sex"
            }],
            "value": 95
        }
    ],
    "type": "lower_limit",
    "value": 80.0
}]

challenger_fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds).result
challenger_fairness_monitor_details.metadata.id




 Waiting for end of monitor instance creation 46eba4d5-8367-406d-9ca9-db53acd7e409 




preparing
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




'46eba4d5-8367-406d-9ca9-db53acd7e409'

## Drift configuration

Enable the drift configuration for both the subscription created with a threshold of 10% and minimal sample as 100 records.

In [50]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == wos_client.monitor_definitions.MONITORS.DRIFT.ID and monitor_instance.entity.target.target_id == pre_prod_subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)


target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=pre_prod_subscription_id

)
parameters = {
    "min_samples": 100,
    "train_drift_model": True,
    "enable_model_drift": False,
    "enable_data_drift": True
}

pre_prod_drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

pre_prod_drift_monitor_details.metadata.id




 Waiting for end of monitor instance creation 46eba4d5-8367-406d-9ca9-db53acd7e409 




preparing
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [51]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == wos_client.monitor_definitions.MONITORS.DRIFT.ID and monitor_instance.entity.target.target_id == challenger_subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)


target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=challenger_subscription_id

)
parameters = {
    "min_samples": 100,
    "train_drift_model": True,
    "enable_model_drift": False,
    "enable_data_drift": True
}

challenger_drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

challenger_drift_monitor_details.metadata.id




 Waiting for end of monitor instance creation ce9a3bc2-bd0e-4c03-b2b1-31dbffa37a6c 




preparing
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




## Drift V2 Configuration

In [52]:
from random import choices

monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == wos_client.monitor_definitions.MONITORS.DRIFT_V2.ID and monitor_instance.entity.target.target_id == pre_prod_subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=pre_prod_subscription_id
)
parameters = {
        "min_samples": 10,
        "max_samples": 1000,
        "train_archive": True,
        "features": {
            "fields": fields,
        },
        "most_important_features": { 
            "fields": choices(fields, k=5)
        },
}
pre_prod_drift_v2_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT_V2.ID,
    target=target,
    parameters=parameters
).result

pre_prod_drift_v2_monitor_details.metadata.id




 Waiting for end of monitor instance creation 9c8ea6c0-9513-458a-a3ed-e0b04f74cbbc 




preparing
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [53]:
from random import choices

monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == wos_client.monitor_definitions.MONITORS.DRIFT_V2.ID and monitor_instance.entity.target.target_id == challenger_subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=challenger_subscription_id
)
parameters = {
        "min_samples": 10,
        "max_samples": 1000,
        "train_archive": True,
        "features": {
            "fields": fields,
        },
        "most_important_features": { 
            "fields": choices(fields, k=5)
        },
}
challenger_drift_v2_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT_V2.ID,
    target=target,
    parameters=parameters
).result

challenger_drift_v2_monitor_details.metadata.id




 Waiting for end of monitor instance creation 0f087b1e-48fc-4a25-bdfd-25f90e2b1dd4 




preparing
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




'0f087b1e-48fc-4a25-bdfd-25f90e2b1dd4'

## Configure Explainability
Finally, we provide OpenScale with the training data to enable and configure the explainability features.

In [54]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=pre_prod_subscription_id
)
parameters = {
    "enabled": True
}
pre_prod_explain_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

pre_prod_explain_monitor_details.metadata.id




 Waiting for end of monitor instance creation 6b2e8d86-3d05-42f6-b32f-6e2a5ce3a294 




preparing
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




'6b2e8d86-3d05-42f6-b32f-6e2a5ce3a294'

In [55]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=challenger_subscription_id
)
parameters = {
    "enabled": True
}
challenger_explain_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

challenger_explain_monitor_details.metadata.id




 Waiting for end of monitor instance creation 2602759f-4180-4fac-93d7-543b3b5aa71c 




preparing
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




'2602759f-4180-4fac-93d7-543b3b5aa71c'

## Enable model risk management (MRM) 

We enable the MRM configuration for both the subscriptions

In [56]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=pre_prod_subscription_id
)
parameters = {
}
pre_prod_mrm_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id='mrm',
    target=target,
    parameters=parameters
).result

pre_prod_mrm_instance_id = pre_prod_mrm_monitor_details.metadata.id




 Waiting for end of monitor instance creation 36867a3c-9170-4b43-8218-041f48822df9 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [57]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=challenger_subscription_id
)
parameters = {
}
challenger_mrm_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id='mrm',
    target=target,
    parameters=parameters
).result

challenger_mrm_instance_id = challenger_mrm_monitor_details.metadata.id




 Waiting for end of monitor instance creation 6a04baa9-baab-49e0-8864-566afa17d24f 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [58]:
wos_client.monitor_instances.show()

00000000-0000-0000-0000-000000000000,active,a9a2e1e9-181e-45bd-9fec-18eea1ddd3e7,subscription,mrm,2024-08-08 00:39:50.234000+00:00,6a04baa9-baab-49e0-8864-566afa17d24f
00000000-0000-0000-0000-000000000000,active,06b4e163-86b5-493d-8e28-8606fb296dca,subscription,mrm,2024-08-08 00:39:42.325000+00:00,36867a3c-9170-4b43-8218-041f48822df9
00000000-0000-0000-0000-000000000000,active,ce36911c-75f6-4f99-b4d2-b71e5d55a802,subscription,model_health,2024-07-03 10:37:23.321000+00:00,097c7a99-a0ac-4a5e-a415-ed733899ed3e
00000000-0000-0000-0000-000000000000,error,a9a2e1e9-181e-45bd-9fec-18eea1ddd3e7,subscription,explainability,2024-08-08 00:38:42.660000+00:00,2602759f-4180-4fac-93d7-543b3b5aa71c
00000000-0000-0000-0000-000000000000,active,c385a4f8-2ef0-441d-baeb-b3f1a11cb3b0,subscription,model_health,2024-06-14 05:38:07.626000+00:00,15ee2b53-f250-4004-a391-b8d65dcdfcff
00000000-0000-0000-0000-000000000000,error,06b4e163-86b5-493d-8e28-8606fb296dca,subscription,explainability,2024-08-08 00:37:39.125000+00:00,6b2e8d86-3d05-42f6-b32f-6e2a5ce3a294
00000000-0000-0000-0000-000000000000,active,ce36911c-75f6-4f99-b4d2-b71e5d55a802,subscription,mrm,2024-07-03 10:37:23.122000+00:00,c3b21f0e-46d6-43bf-bb76-917587d52557
00000000-0000-0000-0000-000000000000,active,1b8ebb3c-e69a-4cc1-9a0a-57424546cab3,subscription,mrm,2024-07-03 04:37:00.338000+00:00,c807c46a-bdd4-493b-9967-47985673990b
00000000-0000-0000-0000-000000000000,active,3fc64ae7-b852-4951-bc65-4e450b473952,subscription,mrm,2024-06-26 04:34:21.284000+00:00,80265b34-0089-46b3-8191-fad7e4557ea4
00000000-0000-0000-0000-000000000000,active,e96278a6-7190-48f3-b8bd-945fa48cfe50,subscription,mrm,2024-07-02 15:33:20.161000+00:00,78f86067-1da9-4f8d-a633-b0ab09dbe020


Note: First 10 records were displayed.


## Patch the integration reference in the pre-prod subscription

In [59]:
payload = [
  {
    "op": "add",
    "path": "/integration_reference",
    "value": {
        "integrated_system_id": integrated_system_id,
        "external_id": openpages_model_id
    }
  }
]

In [60]:
response = wos_client.subscriptions.update(pre_prod_subscription_id, payload)
response.result.to_dict()

{'metadata': {'id': '06b4e163-86b5-493d-8e28-8606fb296dca',
  'crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:subscription:06b4e163-86b5-493d-8e28-8606fb296dca',
  'url': '/v2/subscriptions/06b4e163-86b5-493d-8e28-8606fb296dca',
  'created_at': '2024-08-08T00:28:25.288000Z',
  'created_by': 'cpadmin',
  'modified_at': '2024-08-08T01:21:27.170000Z',
  'modified_by': 'cpadmin'},
 'entity': {'data_mart_id': '00000000-0000-0000-0000-000000000000',
  'service_provider_id': '33397d85-dcae-4703-907d-4668932f7df0',
  'asset': {'asset_id': 'fe7ed7dd-2856-42d6-83ab-51c13079f78d',
   'url': 'https://internal-nginx-svc:12443/ml/v4/models/fe7ed7dd-2856-42d6-83ab-51c13079f78d?space_id=498630dc-3e0b-4470-b44d-583f2a393c0c&version=2020-06-12',
   'name': 'German Credit Risk Model - PreProd',
   'asset_type': 'model',
   'problem_type': 'binary',
   'model_type': 'mllib_3.3',
   'input_data_type': 'structured'},
  'asset_properties': {'training_data_referenc

In [61]:
response = wos_client.subscriptions.update(challenger_subscription_id, payload)
response.result.to_dict()

{'metadata': {'id': 'a9a2e1e9-181e-45bd-9fec-18eea1ddd3e7',
  'crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:subscription:a9a2e1e9-181e-45bd-9fec-18eea1ddd3e7',
  'url': '/v2/subscriptions/a9a2e1e9-181e-45bd-9fec-18eea1ddd3e7',
  'created_at': '2024-08-08T00:28:30.628000Z',
  'created_by': 'cpadmin',
  'modified_at': '2024-08-08T01:21:30.687000Z',
  'modified_by': 'cpadmin'},
 'entity': {'data_mart_id': '00000000-0000-0000-0000-000000000000',
  'service_provider_id': '33397d85-dcae-4703-907d-4668932f7df0',
  'asset': {'asset_id': 'fe7ed7dd-2856-42d6-83ab-51c13079f78d',
   'url': 'https://internal-nginx-svc:12443/ml/v4/models/fe7ed7dd-2856-42d6-83ab-51c13079f78d?space_id=498630dc-3e0b-4470-b44d-583f2a393c0c&version=2020-06-12',
   'name': 'German Credit Risk Model - PreProd',
   'asset_type': 'model',
   'problem_type': 'binary',
   'model_type': 'scikit-learn_0.23',
   'input_data_type': 'structured'},
  'asset_properties': {'training_data_

## Create test data sets from the training data 

In [62]:
test_data_1 = pd_data[1:201]
test_data_1.to_csv("german_credit_risk_test_data_1.csv", encoding="utf-8", index=False)
test_data_2 = pd_data[201:401]
test_data_2.to_csv("german_credit_risk_test_data_2.csv", encoding="utf-8", index=False)
test_data_3 = pd_data[401:601]
test_data_3.to_csv("german_credit_risk_test_data_3.csv", encoding="utf-8", index=False)
test_data_4 = pd_data[601:801]
test_data_4.to_csv("german_credit_risk_test_data_4.csv", encoding="utf-8", index=False)

## Function to upload, evaluate and check the status of the evaluation

This function will upload the test data CSV and trigger the risk evaluation. It will iterate and check the status of the evaluation until its finished with a finite wait duration

In [63]:
def upload_and_evaluate(file_name, mrm_instance_id):
    
    print("Running upload and evaluate for {}".format(file_name))
    import json
    import time
    from datetime import datetime

    status = None
    monitoring_run_id = None
    GET_UPLOAD_AND_EVALUATION_STATUS_RETRIES = 32
    GET_UPLOAD_AND_EVALUATION_STATUS_INTERVAL = 10
    
    if file_name is not None:
        
        response = wos_client.monitor_instances.mrm.evaluate_risk(
                        monitor_instance_id = mrm_instance_id,
                        test_data_set_name = file_name,
                        feedback_data_path = file_name,
                        test_data_path=file_name)
    
        if response.status_code != 202:
            print("Upload and evalaute for {0} failed with error : {1}".format(file_name, response.status_code))
            return
        
        for i in range(GET_UPLOAD_AND_EVALUATION_STATUS_RETRIES):
        
            response = wos_client.monitor_instances.mrm.get_risk_evaluation(
                            monitor_instance_id = mrm_instance_id)
            
            if response.status_code != 200:
                print("Getting status of upload and evalaute for {0} failed with error : {1}".format(file_name, response.status_code))
                return

            response = response.result._to_dict()

            if "metadata" in response and "id" in response["metadata"]:
                monitoring_run_id = response["metadata"]["id"]
            if "entity" in response and "status" in response["entity"]:
                status = response["entity"]["status"]["state"]
            
            if status is not None:
                print(datetime.utcnow().strftime('%H:%M:%S'), status.lower())
                if status.lower() in ["finished", "completed"]:
                    break
                elif "error" in status.lower():
                    print(response)
                    break

            time.sleep(GET_UPLOAD_AND_EVALUATION_STATUS_INTERVAL)

    return status, monitoring_run_id

## Perform Risk Evaluations

We now start performing evaluations of smaller data sets against both the PreProd and Challenger subscriptions

In [64]:
upload_and_evaluate("german_credit_risk_test_data_1.csv", pre_prod_mrm_instance_id)

Running upload and evaluate for german_credit_risk_test_data_1.csv
00:41:09 upload_in_progress
00:41:20 upload_in_progress
00:41:31 running
00:41:42 running
00:41:56 running
00:42:07 running
00:42:18 running
00:42:29 running
00:42:40 finished


('finished', 'b97e7700-22b1-4d77-978b-4afa24d49abd')

In [65]:
upload_and_evaluate("german_credit_risk_test_data_2.csv", pre_prod_mrm_instance_id)

Running upload and evaluate for german_credit_risk_test_data_2.csv
00:42:47 upload_in_progress
00:42:58 running
00:43:09 running
00:43:20 running
00:43:31 finished


('finished', '1513f272-2cb7-4d62-bd0b-098a3106dcab')

In [66]:
upload_and_evaluate("german_credit_risk_test_data_3.csv", pre_prod_mrm_instance_id)

Running upload and evaluate for german_credit_risk_test_data_3.csv
00:43:34 upload_in_progress
00:43:45 running
00:43:56 running
00:44:07 running
00:44:18 finished


('finished', 'cb1905a7-4103-4116-8507-874bdc3f094f')

In [67]:
upload_and_evaluate("german_credit_risk_test_data_4.csv", pre_prod_mrm_instance_id)

Running upload and evaluate for german_credit_risk_test_data_4.csv
00:44:21 upload_in_progress
00:44:32 running
00:44:43 running
00:44:54 running
00:45:05 finished


('finished', '55c28302-3bbf-4a0e-a106-2f632d9dad26')

In [68]:
upload_and_evaluate("german_credit_risk_test_data_1.csv", challenger_mrm_instance_id)

Running upload and evaluate for german_credit_risk_test_data_1.csv
00:45:08 upload_in_progress
00:45:19 upload_in_progress
00:45:30 finished


('finished', 'ab26bdb4-eb7f-4d0d-ad7d-6a99e43dc261')

In [69]:
upload_and_evaluate("german_credit_risk_test_data_2.csv", challenger_mrm_instance_id)

Running upload and evaluate for german_credit_risk_test_data_2.csv
00:45:33 upload_in_progress
00:45:44 finished


('finished', '06f41e46-200b-40b5-8ccb-2a149ff17b0c')

In [70]:
upload_and_evaluate("german_credit_risk_test_data_3.csv", challenger_mrm_instance_id)

Running upload and evaluate for german_credit_risk_test_data_3.csv
00:45:47 upload_in_progress
00:45:58 upload_in_progress
00:46:09 finished


('finished', '533e43b7-e921-4cb3-abc4-913b03936214')

In [71]:
upload_and_evaluate("german_credit_risk_test_data_4.csv", challenger_mrm_instance_id)

Running upload and evaluate for german_credit_risk_test_data_4.csv
00:46:12 upload_in_progress
00:46:23 upload_in_progress
00:46:34 finished


('finished', 'cab790a9-8c34-4d6c-9cf8-08b8adfd8cbd')

## Explore the Model Risk Management UI

Here is a quick recap of what we have done so far.

1. We've deployed two Credit Risk Model to a WML instance that is designated as Pre-Production
2. We've created subscriptions of these two model deployments in OpenScale
3. Configured all monitors supported by OpenScale for these subscriptions
4. We've performed a few risk evaluations against both these susbscription with the same set of test data

Now, please explore the Model Risk Management UI to visualize the results, compare the performance of models, download the evaluation report as PDF. For more information, refer to the Beta Guide section "Work in Watson OpenScale."

Link to OpenScale : https://CLUSTER_URL/aiopenscale/insights

# Promote pre-production model to production 

After you have reviewed the evaluation results of the PreProd Vs Challenger and if you make the decision to promote the Challenger model to Production, the first thing you need to do is to deploy the model into a WML instance that is designated as Production instance

## Deploy model to production WML instance 

In [72]:
PROD_MODEL_NAME="German Credit Risk Model - Prod"
PROD_DEPLOYMENT_NAME="German Credit Risk Model - Prod"

PROD_SPACE_NAME="prod"

In [73]:
wml_client.spaces.list()

# Find and set the default space
space_name=PROD_SPACE_NAME
spaces = wml_client.spaces.get_details()['resources']
space_id = None
for space in spaces:
    if space['entity']['name'] == space_name:
        space_id = space["metadata"]["id"]
if space_id is None:
    space_id = wml_client.spaces.store(
        meta_props={wml_client.spaces.ConfigurationMetaNames.NAME: space_name})["metadata"]["id"]
wml_client.set.default_space(space_id)

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  -----------------------------------------------------------------------  ------------------------
ID                                    NAME                                                                     CREATED
498630dc-3e0b-4470-b44d-583f2a393c0c  wml_preprod_space                                                        2024-08-08T00:19:18.705Z
00969846-29b6-446a-959e-587df21992e0  prod                                                                     2024-08-07T06:33:55.297Z
06873ce9-baf3-46a6-86c4-54c1e3d84429  tutorial_space                                                           2024-08-06T13:29:18.489Z
e8b87647-a0e9-4932-920f-1e8f1d1f383d  tutorial-space                                                           2024-08-06T12:55:07.962Z
832d6bbe-a2ea-451f-8d94-853da7c52b00  FrauDScore                                               

'SUCCESS'

In [74]:
import numpy 
numpy.version.version

import pandas as pd
import json

from pyspark import SparkContext, SQLContext
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier,GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import StringIndexer, VectorAssembler, IndexToString
from pyspark.sql.types import StructType, DoubleType, StringType, ArrayType

from pyspark.sql import SparkSession
from pyspark import SparkFiles

spark = SparkSession.builder.getOrCreate()
pd_data = pd.read_csv("german_credit_data_biased_training.csv", sep=",", header=0)
spark_df = spark.read.csv(path="german_credit_data_biased_training.csv", sep=",", header=True, inferSchema=True)
spark_df.head()

(train_data, test_data) = spark_df.randomSplit([0.9, 0.1], 24)
print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))

si_CheckingStatus = StringIndexer(inputCol='CheckingStatus', outputCol='CheckingStatus_IX')
si_CreditHistory = StringIndexer(inputCol='CreditHistory', outputCol='CreditHistory_IX')
si_LoanPurpose = StringIndexer(inputCol='LoanPurpose', outputCol='LoanPurpose_IX')
si_ExistingSavings = StringIndexer(inputCol='ExistingSavings', outputCol='ExistingSavings_IX')
si_EmploymentDuration = StringIndexer(inputCol='EmploymentDuration', outputCol='EmploymentDuration_IX')
si_Sex = StringIndexer(inputCol='Sex', outputCol='Sex_IX')
si_OthersOnLoan = StringIndexer(inputCol='OthersOnLoan', outputCol='OthersOnLoan_IX')
si_OwnsProperty = StringIndexer(inputCol='OwnsProperty', outputCol='OwnsProperty_IX')
si_InstallmentPlans = StringIndexer(inputCol='InstallmentPlans', outputCol='InstallmentPlans_IX')
si_Housing = StringIndexer(inputCol='Housing', outputCol='Housing_IX')
si_Job = StringIndexer(inputCol='Job', outputCol='Job_IX')
si_Telephone = StringIndexer(inputCol='Telephone', outputCol='Telephone_IX')
si_ForeignWorker = StringIndexer(inputCol='ForeignWorker', outputCol='ForeignWorker_IX')
si_Label = StringIndexer(inputCol="Risk", outputCol="label").fit(spark_df)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_Label.labels)

va_features = VectorAssembler(
inputCols=["CheckingStatus_IX", "CreditHistory_IX", "LoanPurpose_IX", "ExistingSavings_IX",
           "EmploymentDuration_IX", "Sex_IX", "OthersOnLoan_IX", "OwnsProperty_IX", "InstallmentPlans_IX",
           "Housing_IX", "Job_IX", "Telephone_IX", "ForeignWorker_IX", "LoanDuration", "LoanAmount",
           "InstallmentPercent", "CurrentResidenceDuration", "LoanDuration", "Age", "ExistingCreditsCount",
           "Dependents"], outputCol="features")

classifier=GBTClassifier(featuresCol="features")

pipeline = Pipeline(
stages=[si_CheckingStatus, si_CreditHistory, si_EmploymentDuration, si_ExistingSavings, si_ForeignWorker,
        si_Housing, si_InstallmentPlans, si_Job, si_LoanPurpose, si_OthersOnLoan,
        si_OwnsProperty, si_Sex, si_Telephone, si_Label, va_features, classifier, label_converter])

model = pipeline.fit(train_data)
predictions = model.transform(test_data)
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction")
auc = evaluator.evaluate(predictions)

print("Accuracy = %g" % auc)

# Remove existing model and deployment
MODEL_NAME=PROD_MODEL_NAME
DEPLOYMENT_NAME=PROD_DEPLOYMENT_NAME

deployment_details = wml_client.deployments.get_details()
for deployment in deployment_details['resources']:
    deployment_id = wml_client.deployments.get_id(deployment)
    model_id = deployment['entity']['asset']['id']
    if deployment['entity']['name'] == DEPLOYMENT_NAME:
        print('Deleting deployment id', deployment_id)
        wml_client.deployments.delete(deployment_id)
        print('Deleting model id', model_id)
        wml_client.repository.delete(model_id)
wml_client.repository.list_models()

# Save Model
software_spec_uid = wml_client.software_specifications.get_id_by_name("spark-mllib_3.3")
model_props_rf = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME,
    wml_client.repository.ModelMetaNames.DESCRIPTION: MODEL_NAME,
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
    wml_client.repository.ModelMetaNames.TYPE: 'mllib_3.3'
}
published_model_details = wml_client.repository.store_model(model=model, meta_props=model_props_rf, training_data=train_data, pipeline=pipeline)
print(published_model_details)

# List models in the repository
wml_client.repository.list_models()

# Get the model UID
prod_model_uid = wml_client.repository.get_model_id(published_model_details)
prod_model_uid


# Deploy model
wml_deployments = wml_client.deployments.get_details()
prod_deployment_uid = None
for deployment in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        prod_deployment_uid = wml_client.deployments.get_id(deployment)
        break

if prod_deployment_uid is None:
    print("Deploying model...")
    meta_props = {
        wml_client.deployments.ConfigurationMetaNames.NAME: DEPLOYMENT_NAME,
        wml_client.deployments.ConfigurationMetaNames.DESCRIPTION: DEPLOYMENT_NAME,
        wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
    deployment = wml_client.deployments.create(artifact_uid=prod_model_uid, name=DEPLOYMENT_NAME, meta_props=meta_props)
    prod_deployment_uid = wml_client.deployments.get_id(deployment)

print("Model id: {}".format(prod_model_uid))
print("Deployment id: {}".format(prod_deployment_uid))

prod_deployment_uid=wml_client.deployments.get_id(deployment)
prod_deployment_uid

fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
values = [
  ["no_checking",13,"credits_paid_to_date","car_new",1343,"100_to_500","1_to_4",2,"female","none",3,"savings_insurance",46,"none","own",2,"skilled",1,"none","yes"],
  ["no_checking",24,"prior_payments_delayed","furniture",4567,"500_to_1000","1_to_4",4,"male","none",4,"savings_insurance",36,"none","free",2,"management_self-employed",1,"none","yes"],
  ["0_to_200",26,"all_credits_paid_back","car_new",863,"less_100","less_1",2,"female","co-applicant",2,"real_estate",38,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",14,"no_credits","car_new",2368,"less_100","1_to_4",3,"female","none",3,"real_estate",29,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",4,"no_credits","car_new",250,"less_100","unemployed",2,"female","none",3,"real_estate",23,"none","rent",1,"management_self-employed",1,"none","yes"],
  ["no_checking",17,"credits_paid_to_date","car_new",832,"100_to_500","1_to_4",2,"male","none",2,"real_estate",42,"none","own",1,"skilled",1,"none","yes"],
  ["no_checking",33,"outstanding_credit","appliances",5696,"unknown","greater_7",4,"male","co-applicant",4,"unknown",54,"none","free",2,"skilled",1,"yes","yes"],
  ["0_to_200",13,"prior_payments_delayed","retraining",1375,"100_to_500","4_to_7",3,"male","none",3,"real_estate",37,"none","own",2,"management_self-employed",1,"none","yes"]
]

payload_scoring = {"fields": fields,"values": values}
payload = {
    wml_client.deployments.ScoringMetaNames.INPUT_DATA: [payload_scoring]
}
scoring_response = wml_client.deployments.score(prod_deployment_uid, payload)

print('Single record scoring result:', '\n fields:', scoring_response['predictions'][0]['fields'], '\n values: ', scoring_response['predictions'][0]['values'][0])

Number of records for training: 4485
Number of records for evaluation: 515
Accuracy = 0.726321
Deleting deployment id 2a85d08a-ea0b-4778-a76d-49099810f8dc
Deleting model id 38f4cc7c-7231-4703-95bc-af7601819ddd
--  ----  -------  ----  ----------  ----------------
ID  NAME  CREATED  TYPE  SPEC_STATE  SPEC_REPLACEMENT
--  ----  -------  ----  ----------  ----------------
Note: Warnings!! :  Software specification spark-mllib_3.3 specified for the wml_model is deprecated and will be removed in the future. We recommend you use spark-mllib_3.4 instead. For details see Supported Frameworks https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=specifications-supported-deployment-frameworks
{'entity': {'hybrid_pipeline_software_specs': [], 'label_column': 'Risk', 'pipeline': {'id': 'fc900e5c-12c5-4125-ab94-0f99230141ea'}, 'schemas': {'input': [{'fields': [{'metadata': {}, 'name': 'CheckingStatus', 'nullable': True, 'type': 'string'}, {'metadata': {}, 'name': 'LoanDuration', 'nullable': True

In [75]:
wml_client.deployments.get_details(prod_deployment_uid)

Note: Software specification spark-mllib_3.3 is deprecated. Use spark-mllib_3.4 software specification instead when saving a spark model. For details, see https://www.ibm.com/support/producthub/icpdata/docs/content/SSQNUZ_latest/wsj/wmls/wmls-deploy-python-types.html.


{'entity': {'asset': {'id': '8342537b-4586-4a4b-94b5-c218bbdca9d9'},
  'custom': {},
  'deployed_asset_type': 'model',
  'description': 'German Credit Risk Model - Prod',
  'hardware_spec': {'id': 'e7ed1d6c-2e89-42d7-aed5-863b972c1d2b',
   'name': 'S',
   'num_nodes': 1},
  'name': 'German Credit Risk Model - Prod',
  'online': {},
  'space_id': '00969846-29b6-446a-959e-587df21992e0',
  'status': {'inference': [{'url': 'https://cpd-cpd-instance.apps.wos415nfs2672.cp.fyre.ibm.com/ml/v4/deployments/17fcc5ed-2ec1-4bba-a7d2-f0bf3a005cd7/predictions'}],
   'online_url': {'url': 'https://cpd-cpd-instance.apps.wos415nfs2672.cp.fyre.ibm.com/ml/v4/deployments/17fcc5ed-2ec1-4bba-a7d2-f0bf3a005cd7/predictions'},
   'serving_urls': ['https://cpd-cpd-instance.apps.wos415nfs2672.cp.fyre.ibm.com/ml/v4/deployments/17fcc5ed-2ec1-4bba-a7d2-f0bf3a005cd7/predictions'],
   'state': 'ready'}},
 'metadata': {'created_at': '2024-08-08T00:47:14.676Z',
  'description': 'German Credit Risk Model - Prod',
  'id':

## Bind WML machine learning instance as Prod

Watson OpenScale needs to be bound to the Watson Machine Learning instance to capture payload data into and out of the model. If a binding with name "WML Prod" already exists, this code will delete that binding a create a new one.

**Note**: Binding with name `WML Prod` is assumed to be only created by this notebook.

In [76]:
SERVICE_PROVIDER_NAME = 'WML Prod'
SERVICE_PROVIDER_DESCRIPTION = 'Prod service provider created by MRM tutorial'

In [77]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

Deleted existing service_provider for WML instance: f4931b54-2103-40d9-b564-9ba3b1982fdb


In [78]:
added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.WATSON_MACHINE_LEARNING,
        deployment_space_id = space_id,
        operational_space_id = "production",
        credentials=WMLCredentialsCP4D(),
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id
service_provider_id




 Waiting for end of adding service provider 4c6d2d1f-79c9-4a5c-a8ac-e1a0fbc35ab2 




active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------




'4c6d2d1f-79c9-4a5c-a8ac-e1a0fbc35ab2'

In [79]:
added_service_provider_result.to_dict()

{'metadata': {'id': '4c6d2d1f-79c9-4a5c-a8ac-e1a0fbc35ab2',
  'crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:service_provider:4c6d2d1f-79c9-4a5c-a8ac-e1a0fbc35ab2',
  'url': '/v2/service_providers/4c6d2d1f-79c9-4a5c-a8ac-e1a0fbc35ab2',
  'created_at': '2024-08-08T00:47:26.285000Z',
  'created_by': 'cpadmin'},
 'entity': {'name': 'WML Prod',
  'service_type': 'watson_machine_learning',
  'instance_id': '99999999-9999-9999-9999-999999999999',
  'credentials': {'secret_id': 'aac7b217-fea3-4a80-b6ed-027e72e96f24'},
  'operational_space_id': 'production',
  'deployment_space_id': '00969846-29b6-446a-959e-587df21992e0',
  'status': {'state': 'active'}}}

## Remove existing prod subscription

This code removes previous subscription that matches the name `German Credit Risk Model - Prod` as it is expected this subscription is created only via this notebook.

In [80]:
subscriptions = wos_client.subscriptions.list().result.subscriptions
for subscription in subscriptions:
    sub_name = subscription.entity.asset.name
    if sub_name == PROD_MODEL_NAME:
        wos_client.subscriptions.delete(subscription.metadata.id)
        print('Deleted existing subscription for', sub_name)

# Import configuration settings from pre-prod model

With MRM we provide a important feature that lets you copy the configuration settings of your pre-production subscription to the production subscription. To try this out

1. Navigate to Model Monitors view in Insights dashboard of OpenScale
2. Click on the Add to dashboard
3. Select the production model deployment from WML production machine learning provider and click on Configure
4. In Selections saved dialog, click on Configure monitors
4. In Model Governance, click on Begin and choose the same OpenPages model that was linked to the pre-production model and click on Save
5. You will be presented with an dialog about the OpenPages model being associated with another subscription and if you'd like to import settings.
6. Click on Import and confirm

All the configuration settings are now copied into the production subscription


<b>Note: The next set of cells should be executed only after finishing the import settings from the OpenScale dashboard</b>

## Score the production model so that we can trigger monitors

Now that the production subscription is configured by copying the configuration, there would be schedules created for each of the monitors to run on a scheduled basis. 
Quality, Fairness and Mrm will run hourly. Drift will run once in three hours.

For this demo purpose, we will trigger the monitors on-demand so that we can see the model summary dashboard without having to wait the entire hour. 
To do that lets first push some records in the Payload Logging table.

In [81]:
df = pd_data.sample(n=400)
df = df.drop(['Risk'], axis=1)
fields = df.columns.tolist()
values = df.values.tolist()

payload_scoring = {"fields": fields,"values": values}

In [82]:
payload = {
    wml_client.deployments.ScoringMetaNames.INPUT_DATA: [payload_scoring]
}
scoring_response = wml_client.deployments.score(prod_deployment_uid, payload)

print('Single record scoring result:', '\n fields:', scoring_response['predictions'][0]['fields'], '\n values: ', scoring_response['predictions'][0]['values'][0])

Single record scoring result: 
 fields: ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', 'CheckingStatus_IX', 'CreditHistory_IX', 'EmploymentDuration_IX', 'ExistingSavings_IX', 'ForeignWorker_IX', 'Housing_IX', 'InstallmentPlans_IX', 'Job_IX', 'LoanPurpose_IX', 'OthersOnLoan_IX', 'OwnsProperty_IX', 'Sex_IX', 'Telephone_IX', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'] 
 values:  ['0_to_200', 21, 'credits_paid_to_date', 'furniture', 2664, 'less_100', '1_to_4', 2, 'male', 'none', 3, 'car_other', 40, 'none', 'rent', 1, 'skilled', 1, 'none', 'yes', 2.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, [2.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 21.0, 2664.0

In [84]:
subscriptions = wos_client.subscriptions.list().result.subscriptions
for subscription in subscriptions:
    sub_name = subscription.entity.asset.name
    if sub_name == PROD_MODEL_NAME:
        prod_subscription = subscription
prod_subscription_id = prod_subscription.metadata.id
print(prod_subscription_id)

a9a2e1e9-181e-45bd-9fec-18eea1ddd3e7


In [85]:
res = wos_client.subscriptions.get(prod_subscription_id).result.to_dict()
res

{'metadata': {'id': 'a9a2e1e9-181e-45bd-9fec-18eea1ddd3e7',
  'crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:subscription:a9a2e1e9-181e-45bd-9fec-18eea1ddd3e7',
  'url': '/v2/subscriptions/a9a2e1e9-181e-45bd-9fec-18eea1ddd3e7',
  'created_at': '2024-08-08T00:28:30.628000Z',
  'created_by': 'cpadmin',
  'modified_at': '2024-08-08T00:28:32.262000Z',
  'modified_by': 'cpadmin'},
 'entity': {'data_mart_id': '00000000-0000-0000-0000-000000000000',
  'service_provider_id': '33397d85-dcae-4703-907d-4668932f7df0',
  'asset': {'asset_id': 'fe7ed7dd-2856-42d6-83ab-51c13079f78d',
   'url': 'https://internal-nginx-svc:12443/ml/v4/models/fe7ed7dd-2856-42d6-83ab-51c13079f78d?space_id=498630dc-3e0b-4470-b44d-583f2a393c0c&version=2020-06-12',
   'name': 'German Credit Risk Model - PreProd',
   'asset_type': 'model',
   'problem_type': 'binary',
   'model_type': 'scikit-learn_0.23',
   'input_data_type': 'structured'},
  'asset_properties': {'training_data_

In [86]:
time.sleep(10)
prod_pl_data_set_id = None
prod_pl_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=prod_subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if prod_pl_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id: ", prod_pl_data_set_id)
    pl_records_count = wos_client.data_sets.get_records_count(prod_pl_data_set_id)
    print("Number of records in the payload logging table: {}".format(pl_records_count))
    if pl_records_count == 0:
        print("Automatic payload logging did not happen, performing explicit payload logging.")
        wos_client.data_sets.store_records(data_set_id=prod_pl_data_set_id, request_body=[PayloadRecord(
                       scoring_id='mrm_'+str(uuid.uuid4()),
                       request=payload_scoring,
                       response=scoring_response['predictions'][0]
                   )])
        time.sleep(5)
        pl_records_count = wos_client.data_sets.get_records_count(prod_pl_data_set_id)
        print("Number of records in the payload logging table: {}".format(pl_records_count))

Payload data set id:  2f7ba47c-cff2-4897-a27e-55b8d60eaafa
Number of records in the payload logging table: 816


## Fetch all monitor instances

In [87]:
monitor_instances = wos_client.monitor_instances.list(target_target_id=prod_subscription_id).result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == wos_client.monitor_definitions.MONITORS.QUALITY.ID:
        quality_monitor_instance_id = monitor_instance.metadata.id
    if monitor_def_id == wos_client.monitor_definitions.MONITORS.DRIFT.ID:
        drift_monitor_instance_id = monitor_instance.metadata.id
    if monitor_def_id == wos_client.monitor_definitions.MONITORS.DRIFT_V2.ID:
        drift_v2_monitor_instance_id = monitor_instance.metadata.id
    if monitor_def_id == wos_client.monitor_definitions.MONITORS.FAIRNESS.ID:
        fairness_monitor_instance_id = monitor_instance.metadata.id
    if monitor_def_id == 'mrm':
        mrm_monitor_instance_id = monitor_instance.metadata.id
                    
print("Quality monitor instance id - {0}".format(quality_monitor_instance_id))
print("Drift monitor instance id - {0}".format(drift_monitor_instance_id))
print("Drift v2 monitor instance id - {0}".format(drift_v2_monitor_instance_id))
print("Fairness monitor instance id - {0}".format(fairness_monitor_instance_id))
print("MRM monitor instance id - {0}".format(mrm_monitor_instance_id))

Quality monitor instance id - c0c3274d-dc61-4384-84e3-5d4b3b258379
Drift monitor instance id - ce9a3bc2-bd0e-4c03-b2b1-31dbffa37a6c
Drift v2 monitor instance id - 0f087b1e-48fc-4a25-bdfd-25f90e2b1dd4
Fairness monitor instance id - 46eba4d5-8367-406d-9ca9-db53acd7e409
MRM monitor instance id - 6a04baa9-baab-49e0-8864-566afa17d24f


## Run on-demand Quality

### Feedback logging

The code below downloads and stores enough feedback data to meet the minimum threshold so that OpenScale can calculate a new accuracy measurement. It then kicks off the accuracy monitor. The monitors run hourly, or can be initiated via the Python API, the REST API, or the graphical user interface.


In [88]:
feedback_dataset_id = None
feedback_dataset = wos_client.data_sets.list(type=DataSetTypes.FEEDBACK, 
                                                target_target_id=prod_subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result
feedback_dataset_id = feedback_dataset.data_sets[0].metadata.id
if feedback_dataset_id is None:
    print("Feedback data set not found. Please check quality monitor status.")
print(feedback_dataset_id)

64ee830d-acab-4fce-bc0a-7a6c4fa93d48


In [89]:
!rm additional_feedback_data_v2.json
!wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/additional_feedback_data_v2.json

--2024-08-08 06:29:32--  https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/additional_feedback_data_v2.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50890 (50K) [text/plain]
Saving to: ‘additional_feedback_data_v2.json’

additional_feedback 100%[===================>]  49.70K  --.-KB/s    in 0.01s   

2024-08-08 06:29:32 (3.58 MB/s) - ‘additional_feedback_data_v2.json’ saved [50890/50890]



In [90]:
with open('additional_feedback_data_v2.json') as feedback_file:
    additional_feedback_data = json.load(feedback_file)
wos_client.data_sets.store_records(feedback_dataset_id, request_body=additional_feedback_data, background_mode=False).result




 Waiting for end of storing records with request id: 9cc5ec04-fe15-4d85-87a6-083b13796ad1 




active

---------------------------------------
 Successfully finished storing records 
---------------------------------------




In [91]:
time.sleep(5)
feedback_records_count = wos_client.data_sets.get_records_count(data_set_id=feedback_dataset_id)
print('Number of records in the feedback table: ', feedback_records_count)

Number of records in the feedback table:  898


In [92]:
if quality_monitor_instance_id is not None:
    run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id, background_mode=False).result




 Waiting for end of monitoring run 71790afc-05d2-4f4b-be08-f0afc85da02d 




finished

---------------------------
 Successfully finished run 
---------------------------




In [93]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

## Run on-demand Drift

In [94]:
if drift_monitor_instance_id is not None:
    run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_monitor_instance_id, background_mode=False).result




 Waiting for end of monitoring run 52821088-b20b-4756-b67e-a28f0773db97 




finished

---------------------------
 Successfully finished run 
---------------------------




In [95]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_monitor_instance_id)

2024-08-07 09:59:46.400114+00:00,data_drift_magnitude,2080115b-d1d8-4bc5-aa20-56251ca634e5,0.0673076923076923,None,0.1,[],drift,505f727e-fa36-473e-91a5-e7f21cbabfc7,bdbf70f7-e301-497c-a135-c49a4dec9d0b,subscription,ad19393e-2078-4316-ac19-7de0ddfd7937


## Run on-demand Drift V2

In [96]:
if drift_v2_monitor_instance_id is not None:
    run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_v2_monitor_instance_id, background_mode=False).result




 Waiting for end of monitoring run 9c8ea6c0-9513-458a-a3ed-e0b04f74cbbc 




finished

---------------------------
 Successfully finished run 
---------------------------




In [97]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_v2_monitor_instance_id)

2024-08-06 13:03:30.703769+00:00,records_processed,79b5fec3-52e9-4ccb-8d28-a48adcd894e2,408.0,None,None,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,ddf7db84-b9a6-4436-93c4-e386a5adb8d7,ed70ede4-5f54-491c-86c1-9845d7530447,subscription,e4098249-ffec-436c-b76d-0b2ffeec5bf4
2024-08-06 13:03:30.703769+00:00,confidence_drift_score,79b5fec3-52e9-4ccb-8d28-a48adcd894e2,0.1534,None,0.05,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,ddf7db84-b9a6-4436-93c4-e386a5adb8d7,ed70ede4-5f54-491c-86c1-9845d7530447,subscription,e4098249-ffec-436c-b76d-0b2ffeec5bf4
2024-08-06 13:03:30.703769+00:00,records_processed,79b5fec3-52e9-4ccb-8d28-a48adcd894e2,408.0,None,None,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,ddf7db84-b9a6-4436-93c4-e386a5adb8d7,ed70ede4-5f54-491c-86c1-9845d7530447,subscription,e4098249-ffec-436c-b76d-0b2ffeec5bf4
2024-08-06 13:03:30.703769+00:00,confidence_drift_score,79b5fec3-52e9-4ccb-8d28-a48adcd894e2,0.1552,None,0.05,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,ddf7db84-b9a6-4436-93c4-e386a5adb8d7,ed70ede4-5f54-491c-86c1-9845d7530447,subscription,e4098249-ffec-436c-b76d-0b2ffeec5bf4
2024-08-06 13:03:38.611350+00:00,records_processed,7e38094c-c3bf-418a-beb7-7033e7e8568a,408.0,None,None,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,ddf7db84-b9a6-4436-93c4-e386a5adb8d7,ed70ede4-5f54-491c-86c1-9845d7530447,subscription,e4098249-ffec-436c-b76d-0b2ffeec5bf4
2024-08-06 13:03:38.611350+00:00,confidence_drift_score,7e38094c-c3bf-418a-beb7-7033e7e8568a,0.1583,None,0.05,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,ddf7db84-b9a6-4436-93c4-e386a5adb8d7,ed70ede4-5f54-491c-86c1-9845d7530447,subscription,e4098249-ffec-436c-b76d-0b2ffeec5bf4
2024-08-06 13:03:38.611350+00:00,records_processed,7e38094c-c3bf-418a-beb7-7033e7e8568a,408.0,None,None,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,ddf7db84-b9a6-4436-93c4-e386a5adb8d7,ed70ede4-5f54-491c-86c1-9845d7530447,subscription,e4098249-ffec-436c-b76d-0b2ffeec5bf4
2024-08-06 13:03:38.611350+00:00,confidence_drift_score,7e38094c-c3bf-418a-beb7-7033e7e8568a,0.1598,None,0.05,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,ddf7db84-b9a6-4436-93c4-e386a5adb8d7,ed70ede4-5f54-491c-86c1-9845d7530447,subscription,e4098249-ffec-436c-b76d-0b2ffeec5bf4
2024-08-06 13:03:38.821078+00:00,records_processed,10619c86-1908-48ea-9301-0df6fdffad98,408.0,None,None,"['algorithm_used:jensen_shannon', 'computed_on:payload', 'field_type:class', 'field_name:predictedLabel']",drift_v2,ddf7db84-b9a6-4436-93c4-e386a5adb8d7,ed70ede4-5f54-491c-86c1-9845d7530447,subscription,e4098249-ffec-436c-b76d-0b2ffeec5bf4
2024-08-06 13:03:38.821078+00:00,prediction_drift_score,10619c86-1908-48ea-9301-0df6fdffad98,0.0235,None,0.05,"['algorithm_used:jensen_shannon', 'computed_on:payload', 'field_type:class', 'field_name:predictedLabel']",drift_v2,ddf7db84-b9a6-4436-93c4-e386a5adb8d7,ed70ede4-5f54-491c-86c1-9845d7530447,subscription,e4098249-ffec-436c-b76d-0b2ffeec5bf4


Note: First 10 records were displayed.


## Run on-demand Fairness

In [98]:
if fairness_monitor_instance_id is not None:
    run_details = wos_client.monitor_instances.run(monitor_instance_id=fairness_monitor_instance_id, background_mode=False).result




 Waiting for end of monitoring run 46eba4d5-8367-406d-9ca9-db53acd7e409 




running......................
finished

---------------------------
 Successfully finished run 
---------------------------




In [99]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

2024-08-07 09:59:46.400114+00:00,data_drift_magnitude,2080115b-d1d8-4bc5-aa20-56251ca634e5,0.0673076923076923,None,0.1,[],drift,505f727e-fa36-473e-91a5-e7f21cbabfc7,bdbf70f7-e301-497c-a135-c49a4dec9d0b,subscription,ad19393e-2078-4316-ac19-7de0ddfd7937


## Run on-demand MRM

In [100]:
if mrm_monitor_instance_id is not None:
    run_details = wos_client.monitor_instances.run(monitor_instance_id=mrm_monitor_instance_id, triggered_by='user', background_mode=False).result




 Waiting for end of monitoring run 975c4f7d-42dd-4f94-ba48-e7abba54a61e 




running......................
finished

---------------------------
 Successfully finished run 
---------------------------




In [101]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=mrm_monitor_instance_id)

2024-08-08 01:02:45.254000+00:00,tests_passed,be483ae2-da00-4347-a204-88083fbd34fd,1.0,None,None,[],mrm,6a04baa9-baab-49e0-8864-566afa17d24f,975c4f7d-42dd-4f94-ba48-e7abba54a61e,subscription,a9a2e1e9-181e-45bd-9fec-18eea1ddd3e7
2024-08-08 01:02:45.254000+00:00,tests_run,be483ae2-da00-4347-a204-88083fbd34fd,1.0,None,None,[],mrm,6a04baa9-baab-49e0-8864-566afa17d24f,975c4f7d-42dd-4f94-ba48-e7abba54a61e,subscription,a9a2e1e9-181e-45bd-9fec-18eea1ddd3e7
2024-08-08 01:02:45.254000+00:00,tests_skipped,be483ae2-da00-4347-a204-88083fbd34fd,3.0,None,None,[],mrm,6a04baa9-baab-49e0-8864-566afa17d24f,975c4f7d-42dd-4f94-ba48-e7abba54a61e,subscription,a9a2e1e9-181e-45bd-9fec-18eea1ddd3e7
2024-08-08 01:02:45.254000+00:00,tests_failed,be483ae2-da00-4347-a204-88083fbd34fd,0.0,None,None,[],mrm,6a04baa9-baab-49e0-8864-566afa17d24f,975c4f7d-42dd-4f94-ba48-e7abba54a61e,subscription,a9a2e1e9-181e-45bd-9fec-18eea1ddd3e7
2024-08-08 00:46:23.119000+00:00,tests_passed,432cf99d-e6f5-4a3d-bfe4-6ccc50cd1cf7,0.0,None,None,['test_data_set_name:german_credit_risk_test_data_4.csv'],mrm,6a04baa9-baab-49e0-8864-566afa17d24f,cab790a9-8c34-4d6c-9cf8-08b8adfd8cbd,subscription,a9a2e1e9-181e-45bd-9fec-18eea1ddd3e7
2024-08-08 00:46:23.119000+00:00,tests_run,432cf99d-e6f5-4a3d-bfe4-6ccc50cd1cf7,1.0,None,None,['test_data_set_name:german_credit_risk_test_data_4.csv'],mrm,6a04baa9-baab-49e0-8864-566afa17d24f,cab790a9-8c34-4d6c-9cf8-08b8adfd8cbd,subscription,a9a2e1e9-181e-45bd-9fec-18eea1ddd3e7
2024-08-08 00:46:23.119000+00:00,tests_skipped,432cf99d-e6f5-4a3d-bfe4-6ccc50cd1cf7,3.0,None,None,['test_data_set_name:german_credit_risk_test_data_4.csv'],mrm,6a04baa9-baab-49e0-8864-566afa17d24f,cab790a9-8c34-4d6c-9cf8-08b8adfd8cbd,subscription,a9a2e1e9-181e-45bd-9fec-18eea1ddd3e7
2024-08-08 00:46:23.119000+00:00,tests_failed,432cf99d-e6f5-4a3d-bfe4-6ccc50cd1cf7,1.0,None,None,['test_data_set_name:german_credit_risk_test_data_4.csv'],mrm,6a04baa9-baab-49e0-8864-566afa17d24f,cab790a9-8c34-4d6c-9cf8-08b8adfd8cbd,subscription,a9a2e1e9-181e-45bd-9fec-18eea1ddd3e7
2024-08-08 00:45:57.944000+00:00,tests_passed,0050100d-8842-4254-a4dc-9d85a0c616ed,0.0,None,None,['test_data_set_name:german_credit_risk_test_data_3.csv'],mrm,6a04baa9-baab-49e0-8864-566afa17d24f,533e43b7-e921-4cb3-abc4-913b03936214,subscription,a9a2e1e9-181e-45bd-9fec-18eea1ddd3e7
2024-08-08 00:45:57.944000+00:00,tests_run,0050100d-8842-4254-a4dc-9d85a0c616ed,1.0,None,None,['test_data_set_name:german_credit_risk_test_data_3.csv'],mrm,6a04baa9-baab-49e0-8864-566afa17d24f,533e43b7-e921-4cb3-abc4-913b03936214,subscription,a9a2e1e9-181e-45bd-9fec-18eea1ddd3e7


Note: First 10 records were displayed.


## Refresh the model summary of the production subscription in the OpenScale dashboard

This brings us to the end of this demo exercise. Thank you for trying it out.